# Initial Supervised Learning Analysis for the HR Separation (hrsepara) project
## Start with output from initialEDA v0.6
### Identify the correct working directory for source data
* '/data/discovery/hrsepara/core/'
### Identify the correct working directory for the analysis (both HDFS and LFS)
* '/data/discovery/hrsepara/staging/eda' and  '/home/kesj/working/hrsepara/eda/'
## 


In [ ]:
coredir = '/data/discovery/hrsepara/core/'
stgdir1 = '/data/discovery/hrsepara/staging/eda'
stgdir1local = '/home/kesj/work/hrsepara/eda'


In [ ]:
import os,subprocess
import pandas as pd
import numpy as np
%matplotlib inline
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn import preprocessing
from itertools import chain
from random import random

In [ ]:
plt.style.use('fivethirtyeight') # Good looking plots
import seaborn as sns

In [ ]:
# check if the path exists
os.path.exists(stgdir1local)

In [ ]:
def hdfs_path_does_exist(path):
    return subprocess.call(['hdfs','dfs','-ls',path])
    # returns 0 if does_exist; 1 otherwise
    

In [ ]:
# make the remote directory
if not hdfs_path_does_exist(stgdir1):
    !hdfs dfs -mkdir {stgdir1}

In [ ]:
# check if the path exists
if not os.path.exists(stgdir1local):
    # make the local directory
    !mkdir {stgdir1local}
    #'/home/kesj/work/hrsepara/eda'


%pwd

In [ ]:
%cd {stgdir1local}

In [ ]:
#define the currentworking directory:
cwd = os.path.abspath(os.curdir)
print cwd

In [ ]:
def summarize_dataframe2(df,show_example=False,verbose=False):
    nrow = len(df)
    summary_df = pd.DataFrame(columns = ['Column','datatype','nmissing','arity','accepted values'])
    len_df = len(summary_df)
    for col in df.columns:
        nmiss = nrow - df[col].value_counts().sum()
        narity = len(df[col].unique())
        if show_example:
            print col, df[col].dtype,nmiss, "\t", narity,":\t", df[col].ix[8320]
        elif verbose:
            print col, df[col].dtype,nmiss, "\t", narity
        accept_val = None
        if narity < 20:
            accept_val = df[col].unique()
        summary_df.loc[len_df] = [col,df[col].dtype,nmiss,narity,accept_val]
        len_df+=1
    # assing fraction of missing
    summary_df['x_missing'] = summary_df['nmissing']/float(nrow)
    
    return summary_df

In [ ]:
def create_factorized_df(input_df,column_name):
    tmp_df = pd.get_dummies(input_df[column_name],prefix=column_name)
    # now drop the largest category
    lgst_category = input_df[column_name].value_counts().index[0]
    #print tmp_df.shape, tmp_df.columns
    #print lgst_category
    base_category = column_name+'_'+str(lgst_category)
    tmp_df.drop(base_category,axis=1,inplace=True)
    print tmp_df.shape, base_category
    return tmp_df,base_category
    

## READ the data May 19, 2015
* use employee_dataframe8.tsv --> contains text fields including dates for HIRE_DT..
* use employee_df_[1,2].csv --> contains just numeric data.
    - employee_df_1 has 180 columns:
        + Historical data there 
        + max value of EXT_FUNC_ID_SFI and FUNC_ID_SFI (represents missing ) changed to 2000.
        + median values used to fill missing values for float columns
        + COMPANY dropped
        + 'LOC_TYPE_DESCR_SFI','GRADE','LOC_STATE','JOB_FAMILY' label encoded
        + ANNUAL_RT droped in lieu of SAL1
        + BOX* kept; RATE*  dropped
        + PTFTCNT* kept; PARTFULL dropped
        + FTCNT* kept; FULLPART dropped
    - keep only the most recent historical data
        


In [ ]:
empl_df = pd.read_csv('employee_df_1.csv')
empl_df.drop('Unnamed: 0', axis=1,inplace=True)
empl_df.head()

In [ ]:
new_read_dict = {'KEY':np.str,'LOCATION':np.str,'EEO1CODE':np.str,'SKEY':np.str,'JOBCODE':np.str,'EMPL_CLASS':np.str, 
                        'COMPANY':np.str,'EXT_FUNC_ID_SFI':np.str,'FUNC_ID_SFI':np.str,
                          'DIVISION_CODE_SFI':np.str,'JOB_FAMILY':np.str,'JOB_FUNCTION':np.str,'ACTRES1':np.str,
                          'ACTRES2':np.str,'ACTRES3':np.str,'ACTRES4':np.str,'ACTRES5':np.str,'ACTRES6':np.str,
                          'ACTRES7':np.str,'ACTRES8':np.str,'ACTRES9':np.str,'ACTRES10':np.str,'zip5':np.str}
emplfull = pd.read_csv('employee_dataframe8.tsv',sep='\t',dtype={'KEY':np.str,'SKEY':np.str,'zip5':np.str})#,dtype=new_read_dict)
print emplfull.shape
emplfull.head()

In [ ]:
## So what columns are present in empl_df? (numerics)
sdf = summarize_dataframe2(empl_df)

In [ ]:
sdf

In [ ]:
len(emplfull.JOBCODE.unique())

In [ ]:
[ a for a in empl_df.columns if a.startswith('FULL')]

In [ ]:
empl_df.plot(kind='scatter',x='FUNC_ID_SFI',y='EXT_FUNC_ID_SFI')
print empl_df[['FUNC_ID_SFI','EXT_FUNC_ID_SFI']].corr()

# This data is now ready to use as input into scikitLearn as a matrix
The standard notation is to define this as X and save it as a float type
Don't forget to create a response variable, y

### I will begin by looking to see if there are columns that have strong correlations.


In [ ]:
full_empl_df_correlation = empl_df.corr()

In [ ]:
arank = full_empl_df_correlation.apply(np.argsort, axis=1)
ranked_cols = full_empl_df_correlation.columns.to_series()[arank.values[:,::-1][:,:2]]
new_frame = pd.DataFrame(ranked_cols,index=full_empl_df_correlation.index)
new_frame

In [ ]:
print full_empl_df_correlation.loc['SAL9','SAL10']
empl_df.plot(kind='scatter',x='SAL9',y='SAL10',alpha=0.3)

### List collumns that are highly correlated (over thresh = 0.95) 

In [ ]:
xcol = new_frame[0].values
ycol = new_frame[1].values
for id in xrange(0,len(xcol)):
    #print xcol[id],ycol[id]
    max_pair_corr = full_empl_df_correlation.loc[xcol[id],ycol[id]]
    if max_pair_corr > 0.95:
        print max_pair_corr, xcol[id],ycol[id]
#full_empl2_df_correlation.loc[new_frame[0],new_frame[1]]

In [ ]:
print len(full_empl_df_correlation), len(empl_df.columns)
set(empl_df.columns) - set(full_empl_df_correlation.index)

In [ ]:
[x for x in empl_df.FUNC_ID_SFI.unique() if x > 1000]

In [ ]:
pd.crosstab(empl_df.RELOCATE_ALL_SFI,empl_df.RELO_STATE_CNT_SFI)

### look at relationship between Tenure_year and Age_year


In [ ]:
empl_df.plot(kind='scatter', x = 'Tenure_years',y = 'Age_years', alpha=0.3)

#### how do the various tenures depend on Box10 values?


In [ ]:
my_range = [empl_df.Tenure_years.min(),empl_df.Tenure_years.max()]
nbins = 40
empl_df[empl_df.BOX10==0].Tenure_years.hist(normed=True,range=my_range,bins=nbins,label='BOX10 = 0')
empl_df[empl_df.BOX10!=0].Tenure_years.hist(normed=True,range=my_range,bins=nbins,alpha=0.8,label='BOX10 >0')
#empl2[empl2.BOX10==1].Tenure_years.hist(normed=True,alpha=0.8,range=my_range,bins=nbins)
#empl2[empl2.BOX10==2].Tenure_years.hist(normed=True,alpha=0.6,range=my_range,bins=nbins)
#empl2[empl2.BOX10==3].Tenure_years.hist(normed=True,alpha=0.4,range=my_range,bins=nbins)
plt.xlabel('Tenure (years)')
plt.legend()
plt.ylabel('Normalized Counts')
print " so BOX10 values of 0 are overwhelmingly linked to low tenure (i.e. under 10 years)"

### Further refine/remove columns because of this discrepancy/lack of historical data

In [ ]:
col_list = empl_df.columns.tolist()
col_list

In [ ]:
empl_df[['MERIT1','MERIT10']].describe()#hist(bins=150)
#empl_df.MERIT1.hist(bins=150,alpha=0.8)

In [ ]:
empl_df[['SAL1','SAL10']].describe()

### clean-up historical data

In [ ]:
three_cols = [x for x in col_list if x.endswith('CNT3')]
five_cols = [x for x in col_list if x.endswith('CNT5')]
ten_cols = [x for x in col_list if x.endswith('CNT10')]
# drop all BOX, MERIT, SAL, PERF and add back in those from last year
to_drop_cols = [x for x in col_list if x.startswith('PERF')]
to_drop_cols+= [x for x in col_list if x.startswith('SAL')]
to_drop_cols += [x for x in col_list if x.startswith('MERIT')]
to_drop_cols += [x for x in col_list if x.startswith('BOX')]
to_drop_cols+=three_cols
to_drop_cols+=five_cols
to_drop_cols+=ten_cols
to_add_cols = ['BOX1','SAL1','MERIT1','PERF1']
empl = empl_df.copy()
empl.drop(to_drop_cols,inplace=True,axis=1)
empl[to_add_cols] = empl_df[to_add_cols].copy()
print empl.shape ## this is the same as if I read in employee_df_2.csv

In [ ]:
### get time_sensitive measures
temporal_cols = [x for x in empl.columns if x.endswith('MOS')]
#temporal_cols.append('Age_years')
#temporal_cols.append('Tenure_years')
temporal_cols#.tolist()

## construct a simple df of employee dates (use timestamps)

In [ ]:
[a for a in emplfull.columns if a.startswith('BIR')]

In [ ]:
empl_dates=pd.DataFrame()
empl_dates[['hire_tstmp','term_tstmp','birth_tstmp']] = emplfull[['HIRE_DT','TERMINATION_DT','BIRTHDATE']].apply(lambda x: pd.to_datetime(x))
#['hire_month'] = emplxtra_df['HIRE_DT'].apply(lambda x: int(str(x)[5:7])
#emplxtra_df['hire_month'].hist(bins=12,color='darkorchid')
#plt.xlabel('Month of Hire')
#plt.ylabel('counts')

In [ ]:
## create procedure for time selections.
def split_on_time(dates_df, split_tstmp):
    #split_tstmp = pd.to_datetime(split_date_str)
    #print split_tstmp
    
    before_idx = dates_df[(dates_df.hire_tstmp < split_tstmp)].index
    after_idx = list(set(dates_df.index) -set(before_idx))
    print len(after_idx), len(dates_df)
    print "Splitting on {0} amounts to a hold-out fraction of {1}".format(split_tstmp,len(after_idx)/float(len(dates_df)))
    return before_idx, after_idx
                                                                        
    

# Begin the Prediction(s)
* first need to create a means to hold out current data
* also revise time-sensitive data for relative date

In [ ]:
## get date information
bidx,aidx = split_on_time(empl_dates,'2005-01-01')

In [ ]:
temporal_cols


In [ ]:
def create_time_split(df,dates_df,split_date_str,target_col,time_columns,tgt_type=np.float,omit_columns=None):
    """Inputs: employee data_frame (df)
               dates_data_frame (dates_df) same length & index as df. has timestamps of hire, term and birth dates.
               split_date_str --> string of format 'YYYY-MM-DD' to split into train & test dfs
               time_columns == list of columns that are time dependent (age, tenure, MOS that need to be updated in training set)
               omit_columns =  list of columns to exclude from analysis
               
    """
    # create timestamp based upon split-date_str
    split_tstmp = pd.to_datetime(split_date_str)
    # obtain indices corresponding to before and after the split timestamp
    before_idx, after_idx = split_on_time(dates_df, split_tstmp)
    # drop unnecessary columns
    if omit_columns is not None:
        df.drop(omit_columns,axis=1,inplace=True)
    
    
    # identify the set of training data that needs time adjustment -- i.e. if date is 
    indices_to_fix = dates_df[(dates_df.hire_tstmp< split_tstmp) & (dates_df.term_tstmp >= split_tstmp)].index
    print "{0} indices need time adjustment".format(len(indices_to_fix))
    
    # create test and training sets.
    train_df = df.ix[before_idx].copy()
    test_df = df.ix[after_idx].copy()
    
    # correct status/sep_status, retired, etc. from training df
    train_df.loc[indices_to_fix,['status','sep_status','retired']]= 0 # reset to zero because term date is after split_tstmp
    # recalculate Age_years & Tenure_years
    ## now calculate age at termination date
    age_years = (split_tstmp - dates_df['birth_tstmp'])/np.timedelta64(1,'Y')
    tenure_years = (split_tstmp - dates_df['hire_tstmp'])/np.timedelta64(1,'Y')
    train_df['Age_years'] = age_years[before_idx]
    train_df['Tenure_years'] = tenure_years[before_idx]
    
    # correct time-sensitve columns --> concern is for recent job changes.
    """# assume today is '01-01-2015'
    today_tstmp = pd.to_datetime('2015-01-01')
    months_ago = (today_tstmp - split_tstmp)/np.timedelta64(1,'M')
    print months_ago
    for tcol in time_columns:
        print tcol
        if tcol == 'TELE_MOS':
    """        
    characteristic_cols = ['status','sep_status','retired']
    
    y_train = train_df[target_col].as_matrix().astype(tgt_type)
    #other_to_drop = characteristic_cols.remove(target_col[0])
    tdf = train_df.drop(characteristic_cols,axis=1).copy()
    feature_names = tdf.columns.tolist() # save the features_names
    X_train = tdf.as_matrix().astype(np.float)
    y_test = test_df[target_col].as_matrix().astype(tgt_type)
    tdf = test_df.drop(characteristic_cols,axis=1).copy()
    X_test = tdf.as_matrix().astype(np.float)
    
    return (X_train,y_train,X_test,y_test,feature_names,indices_to_fix)

In [ ]:
X,y,Xtest,ytest,feature_names,fix_idx = create_time_split(empl,empl_dates,'2005-01-01','status',temporal_cols)

In [ ]:
## check balance of status:
print "Training set has {0}".format(sum(y)/float(len(X)))
print "Test set has {0}".format(sum(ytest)/float(len(Xtest)))

## Now run through a RF

In [ ]:
from sklearn import cross_validation
from sklearn import ensemble

In [ ]:
# create a K-fold CV set of models (assumes roughly balanced
def kfold_cv(X, y, clf_class, shuffle=True, n_folds=10, **kwargs):
    k_fold = cross_validation.KFold(len(y), n_folds=n_folds, shuffle=shuffle)
    #y_pred = y.copy()
    kf_fits = []
    for ii, jj in k_fold:
        X_train, X_test = X[ii], X[jj]
        y_train = y[ii]
        clf = clf_class(**kwargs)
        clf.fit(X_train,y_train)
        kf_fits.append(clf)
        #y_pred[jj] = clf.predict(X_test)
    return k_fold,kf_fits

In [ ]:
%%time 
CVscores = []
rfc_500trees, rfc_500fits = kfold_cv(X,y, ensemble.RandomForestClassifier,n_estimators =500, n_jobs=50)
for idx,(ii, jj) in enumerate(rfc_500trees):
    #print len(ii), len(jj)#, idx
    cv_train, cv_test = X[ii],X[jj]
    cv_score = rfc_500fits[idx].score(cv_test,y[jj])
    print idx,"\t",cv_score
    CVscores.append(cv_score)
print "Average CV score is {0}".format(np.mean(CVscores))


In [ ]:
# function for combining an ensemble of estimators (i.e. one generated by KFold)
def combine_rf_estimators(rf_a,rf_b):
    rf_a.estimators_ += rf_b.estimators_
    rf_a.n_estimators = len(rf_a.estimators_)
    return rf_a

In [ ]:
rfc_combo = reduce(combine_rf_estimators, rfc_500fits)
rfc_combo.score(X,y)

## some functions for plotting results.

In [ ]:
# plot ROC curve (requires probability being predicted)
def plot_roc_curve(target_test, target_predicted_proba):
    fpr, tpr, thresholds = metrics.roc_curve(target_test, target_predicted_proba[:, 1])
    
    roc_auc = metrics.auc(fpr, tpr)
    # Plot ROC curve
    plt.plot(fpr, tpr, label='ROC curve (area = %0.3f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')  # random predictions curve
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate or (1 - Specifity)')
    plt.ylabel('True Positive Rate or (Sensitivity)')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    return roc_auc

# plot FI bar
# function to plot feature_importances for RF
def plotFI(forest,featureNames=[],cname='steelblue'):
    featureImportances=forest.feature_importances_
    # sort the importances from biggest to least
    indices = np.argsort(featureImportances)[::-1]
    estimators = forest.estimators_
    # calculate the variance over the forest 
    
    std = np.std([tree.feature_importances_ for tree in estimators],axis=0)
    # print summary statement
    nfeatures = len(featureImportances)
    print("Number of Features: %d" % (nfeatures))
    print("Number of Trees: %d" %(len(estimators)))
    
    #print featureNames
    if len(featureNames)==0:
        featureNames = map(str,indices)
    
    fN2 = [featureNames[a] for a in indices]
    print("Feature ranking:")

    for f in range(len(indices)):
        print("%d. feature %d=%s (%f)" % (f + 1, indices[f], featureNames[indices[f]],featureImportances[indices[f]]))

    # Plot the feature importances of the forest
    # define a cutoff in terms of feature_importance
    if nfeatures <= 30:
        kfeatures = nfeatures # keep all if smaller than 30
    else:
        kfeatures = 30
        
    kindices = indices[:kfeatures]
    plt.title("Feature importances")
    plt.barh(range(len(kindices)), featureImportances[kindices],
       color=cname, xerr=std[kindices], align="center",ecolor='k')#,lw=2)
    plt.yticks(range(len(kindices)),fN2)
    #grid(True)
    c1 = 'value'
    c2 = 'std'
    tdata = np.vstack([featureImportances[indices],std[indices]])
    df = pd.DataFrame(data = tdata.T,index=fN2,columns=[c1,c2])
    return df
    

In [ ]:

fi500a = plotFI(rfc_combo,feature_names,'indianred')

## Test accuracy by applying to set within time-slice that had to be changed. 
###(i.e. term_date/hire_date were within today-slice date range)

In [ ]:
# create accuracy test set:

df_acc = empl.ix[fix_idx].copy()
print len(df_acc)
y_acc = df_acc.status.as_matrix().astype(np.float)
df_acc.drop(['status','sep_status','retired'],inplace=True,axis=1)
X_acc = df_acc.as_matrix().astype(np.float)
rfc_combo.score(X_acc,y_acc)

In [ ]:
def create_accuracy_set(df,fix_idx,tgt_col,char_col=['status','sep_status','retired']):
    df_acc = df.ix[fix_idx].copy()
    print len(df_acc)
    y_acc = df_acc[tgt_col].as_matrix().astype(np.float)
    df_acc.drop(char_col,inplace=True,axis=1)
    X_acc = df_acc.as_matrix().astype(np.float)
    return (X_acc,y_acc)


## Apply to training data

In [ ]:
rfc_combo.score(Xtest,ytest)

In [ ]:
from sklearn import metrics

In [ ]:
auc_score = plot_roc_curve(ytest,rfc_combo.predict_proba(Xtest))

In [ ]:
ytest_pred = rfc_combo.predict(Xtest)
rfc_conf_matrix_tenure = metrics.confusion_matrix(ytest,ytest_pred)
sns.heatmap(rfc_conf_matrix_tenure, annot=True,  fmt='')
plt.title('Random Forest Classifier Confusion Matrix')


In [ ]:
metrics.matthews_corrcoef(ytest,ytest_pred)

In [ ]:
def calculate_metrics(model,y_true,myX):
    # predict
    y_pred = model.predict(myX)
    y_pred_proba = model.predict_proba(myX)
    accuracy = model.score(myX,y_true)
    
    mcc = metrics.matthews_corrcoef(y_true,y_pred)
    #conf_matrix = metrics.confusion_matrix(y_true,y_pred)
    #sns.heatmap(conf_matrix, annot=True,  fmt='')
    #plt.title('Confusion Matrix')
    auc_score = plot_roc_curve(y_true,y_pred_proba)
    return([accuracy,mcc, auc_score])

In [ ]:
calculate_metrics(rfc_combo,ytest,Xtest)

In [ ]:
calculate_metrics(rfc_combo,y_acc,X_acc)

## now do this for a series of different time slices 
(just for fun)

* go in 1 year increments from 1998 to 2014
* calc scores for Accuracy set and Future set
* retain fraction of hold out data, date, combo score, auc_score


In [ ]:
time_slices_range = ['2000-01-01','2005-01-01','2010-01-01']#['1995-01-01','2000-01-01','2005-01-01','2010-01-01']#,'

In [ ]:
measures = []
dsets = []
models = []
for tslice in time_slices_range:
    print tslice
    X,y,Xtest,ytest,feature_names,fix_idx = create_time_split(empl,empl_dates,tslice,'status',temporal_cols)
    #extract accuracy set
    X_acc,y_acc = create_accuracy_set(empl,fix_idx,'status')
    # do a single RF
    single_rfc_500trees = ensemble.RandomForestClassifier(n_estimators=500,n_jobs=50)
    rfc_mdl = single_rfc_500trees.fit(X,y) # fit on training data
    models.append(rfc_mdl)
    dsets.append([X,Xtest,X_acc,y,ytest,y_acc])
    m_acc_a,m_acc_m,m_acc_r = calculate_metrics(rfc_mdl,y_acc,X_acc)
    m_new_a,m_new_m,m_new_r = calculate_metrics(rfc_mdl,ytest,Xtest)
    measures.append([m_acc_a,m_acc_m, m_acc_r, m_new_a, m_new_m, m_new_r])
    

In [ ]:
measures

In [ ]:
print len(dsets)
len(dsets[0])
#conf_matrix = metrics.confusion_matrix(y_true,y_pred)
    #sns.heatmap(conf_matrix, annot=True,  fmt='')
    #plt.title('Confusion Matrix')

In [ ]:
idx= 0
conf_matrix = metrics.confusion_matrix(dsets[idx][4],models[idx].predict(dsets[idx][1]))
sns.heatmap(conf_matrix,annot=True,fmt='')
plt.title('confusion matrix (new)')

In [ ]:
idx= 0
conf_matrix = metrics.confusion_matrix(dsets[idx][5],models[idx].predict(dsets[idx][2]))
sns.heatmap(conf_matrix,annot=True,fmt='')
plt.title('confusion matrix (accuracy)')

In [ ]:
idx= 2
conf_matrix = metrics.confusion_matrix(dsets[idx][4],models[idx].predict(dsets[idx][1]))
sns.heatmap(conf_matrix,annot=True,fmt='')
plt.title('confusion matrix (new)')

In [ ]:
idx= 2
conf_matrix = metrics.confusion_matrix(dsets[idx][5],models[idx].predict(dsets[idx][2]))
sns.heatmap(conf_matrix,annot=True,fmt='')
plt.title('confusion matrix (accuracy)')

## Okay so now I can do the following:
0. monitor the parameters for a given model and dataset
1. optimize for accuracy in my model
2. consider using stratified kfold if classes become unbalanced
3. create a version of the above that takes a window of time instead of a strict split into 2 datasets

# push the data/parameters into a small df
rfmodels_df = pd.DataFrame(data=measures,index=time_slices_range,columns=['Acc_in','MCC_in','AUC_in','Acc_post','MCC_post','AUC_post'])
rfmodels_df.head()

In [ ]:
ds_size = []
for b in xrange(0,len(dsets)):
    ds_size.append([len(dsets[b][a]) for a in xrange(0,len(dsets))])

ds_size=np.array(ds_size) # convert to an array


In [ ]:
ds_size

In [ ]:
rfmodels_df = pd.DataFrame(data=measures,index=time_slices_range,columns=['Acc_in','MCC_in','AUC_in','Acc_post','MCC_post','AUC_post']) 
rfmodels_df.head()

rfmodels_df['Train_size'] = ds_size[:,0]
rfmodels_df['in_size' ] = ds_size[:,1]
rfmodels_df['post_size'] = ds_size[:,2]

In [ ]:
rfmodels_df.head()

### also want to to get balance in class sizes

In [ ]:
## create procedure for time selections.
def divide_by_start_end_time(dates_df, start_tstmp, end_tstmp):
    # eliminate those who left before start_tstmp
    drop_initial_dates_idx = dates_df[(dates_df.term_tstmp < start_tstmp)].index
    drop_after_dates_idx = dates_df[(dates_df.hire_tstmp >= end_tstmp)].index
    keep_range_idx = list(set(dates_df.index)-set(drop_initial_dates_idx)-set(drop_after_dates_idx))
    
    #before_idx = dates_df[(dates_df.hire_tstmp < split_tstmp)].index
    
    print len(keep_range_idx), len(dates_df), len(drop_initial_dates_idx),len(drop_after_dates_idx)
    print "Desired range to keep is between {0} and {1}.".format(start_tstmp,end_tstmp)
    print "This amounts to a training set of {0} elements and a test set of {1}".format(len(keep_range_idx),len(drop_after_dates_idx))
        
    return keep_range_idx, drop_after_dates_idx.tolist()

In [ ]:
train_idx, after_idx = divide_by_start_end_time(empl_dates,pd.to_datetime('2000-01-01'),pd.to_datetime('2005-01-01'))

In [ ]:
len(train_idx), len(after_idx), len(empl)

In [ ]:
empl_dates.ix[after_idx].term_tstmp.min()

In [ ]:
ed = pd.to_datetime('2010-01-01')
st = pd.to_datetime('2005-01-01')
diff_in_years = int(round((ed-st).total_seconds()/(60.*60.*24.*365.25),0) )

In [ ]:
from datetime import date

def add_years(d, years):
    """Return a date that's `years` years after the date (or datetime)
    object `d`. Return the same calendar date (month and day) in the
    destination year, if it exists, otherwise use the following day
    (thus changing February 29 to March 1).

    """
    try:
        return d.replace(year = d.year + years)
    except ValueError:
        return d + (date(d.year + years, 1, 1) - date(d.year, 1, 1))

In [ ]:
def create_time_segment(df,dates_df,start_date_str,end_date_str,target_col,time_columns,tgt_type=np.float,omit_columns=None):
    """Inputs: employee data_frame (df)
               dates_data_frame (dates_df) same length & index as df. has timestamps of hire, term and birth dates.
               start_date_str --> string of format 'YYYY-MM-DD' to split into train & test dfs
               end_date_str --> string of format 'YYYY-MM-DD' to split into train & test dfs
               time_columns == list of columns that are time dependent (age, tenure, MOS that need to be updated in training set)
               omit_columns =  list of columns to exclude from analysis
               
    """
    # convert to datetime.date (let pandas deal with tstmp comparisons)
    MAX_DATE = date(2014,12,31) # current max date
    # create timestamp based upon split-date_str
    #start_tstmp = pd.to_datetime(start_date_str)
    #end_tstmp = pd.to_datetime(end_date_str)
    start_dt = date(int(start_date_str[:4]),int(start_date_str[5:7]),int(start_date_str[-2:]))
    end_dt = date(int(end_date_str[:4]),int(end_date_str[5:7]),int(end_date_str[-2:]))
    # obtain indices corresponding to before and after the split timestamp
    train_idx, after_idx = divide_by_start_end_time(dates_df,start_dt,end_dt)
    # further split 2nd set to have same time length (if possible)
    range_in_years = int(np.round((end_dt - start_dt).total_seconds()/(60.*60.*24.*365.25),0))
    # add_years
    final_dt = add_years(end_dt,range_in_years)
    if final_dt > MAX_DATE:
        final_dt = MAX_DATE
    
    test_idx, holdout_idx = divide_by_start_end_time(dates_df,end_dt,final_dt)
    
    
    
    #before_idx, after_idx = split_on_time(dates_df, start_tstmp)
    #before_idx2, after_idx2 = split_on_time(dates_df, end_tstmp)
    
    # drop unnecessary columns
    if omit_columns is not None:
        df.drop(omit_columns,axis=1,inplace=True)
    
    
    # identify the set of training data that needs time adjustment -- i.e. if date is 
    indices_to_fix = dates_df[(dates_df.hire_tstmp< end_dt) & (dates_df.term_tstmp >= end_dt)].index
    print "{0} indices need time adjustment".format(len(indices_to_fix))
    
    # create test and training sets.
    train_df = df.ix[train_idx].copy()
    test_df = df.ix[test_idx].copy()
    holdout_df = df.ix[holdout_idx].copy()
    # calculate fraction in each class
    x_separated_train = train_df[target_col].value_counts().ix[1]/float(len(train_df))
    x_separated_test = test_df[target_col].value_counts().ix[1]/float(len(test_df))
    x_separated_holdout = holdout_df[target_col].value_counts().ix[1]/float(len(holdout_df))
    # correct status/sep_status, retired, etc. from training df
    train_df.loc[indices_to_fix,['status','sep_status','retired']]= 0 # reset to zero because term date is after split_tstmp
    # recalculate Age_years & Tenure_years
    ## now calculate age at termination date
    age_years = (end_dt - dates_df['birth_tstmp'])/np.timedelta64(1,'Y')
    tenure_years = (end_dt - dates_df['hire_tstmp'])/np.timedelta64(1,'Y')
    train_df['Age_years'] = age_years[train_idx]
    train_df['Tenure_years'] = tenure_years[train_idx]
    
    # correct time-sensitve columns --> concern is for recent job changes.
    """# assume today is '01-01-2015'
    today_tstmp = pd.to_datetime('2015-01-01')
    months_ago = (today_tstmp - split_tstmp)/np.timedelta64(1,'M')
    print months_ago
    for tcol in time_columns:
        print tcol
        if tcol == 'TELE_MOS':
    """        
    characteristic_cols = ['status','sep_status','retired']
    
    y_train = train_df[target_col].as_matrix().astype(tgt_type)
    #other_to_drop = characteristic_cols.remove(target_col[0])
    tdf = train_df.drop(characteristic_cols,axis=1).copy()
    feature_names = tdf.columns.tolist() # save the features_names
    X_train = tdf.as_matrix().astype(np.float)
    y_test = test_df[target_col].as_matrix().astype(tgt_type)
    tdf = test_df.drop(characteristic_cols,axis=1).copy()
    X_test = tdf.as_matrix().astype(np.float)
    frac_class = [x_separated_train, x_separated_test, x_separated_holdout]
    
    return (X_train,y_train,X_test,y_test,feature_names,indices_to_fix,frac_class)

In [ ]:
trX,tr_y,teX,te_y,featuresCols,indx_to_Fix, frac_classes = create_time_segment(empl,empl_dates,'2004-01-01','2008-01-01','status',temporal_cols)
    #df,dates_df,start_date_str,end_date_str,target_col,time_columns,tgt_type=np.float,omit_columns=None):

In [ ]:
len(empl_dates[(empl_dates.hire_tstmp < date(2005,1,1)) & (empl_dates.term_tstmp >= date(2005,1,1))].index), len(train_idx)

In [ ]:
frac_classes

In [ ]:
def create_time_slice(df,dates_df,start_date_str,end_date_str,target_col,time_columns,tgt_type=np.float,omit_columns=None):
    """Inputs: employee data_frame (df)
               dates_data_frame (dates_df) same length & index as df. has timestamps of hire, term and birth dates.
               start_date_str --> string of format 'YYYY-MM-DD' to split into train & test dfs
               end_date_str --> string of format 'YYYY-MM-DD' to split into train & test dfs
               time_columns == list of columns that are time dependent (age, tenure, MOS that need to be updated in training set)
               omit_columns =  list of columns to exclude from analysis
               
    """
    # convert to datetime.date (let pandas deal with tstmp comparisons)
    MAX_DATE = date(2014,12,31) # current max date
    # create timestamp based upon split-date_str
    #start_tstmp = pd.to_datetime(start_date_str)
    #end_tstmp = pd.to_datetime(end_date_str)
    start_dt = date(int(start_date_str[:4]),int(start_date_str[5:7]),int(start_date_str[-2:]))
    end_dt = date(int(end_date_str[:4]),int(end_date_str[5:7]),int(end_date_str[-2:]))
    # obtain indices corresponding to before and after the split timestamp
    train_idx, test_idx = divide_by_start_end_time(dates_df,start_dt,end_dt)
    range_in_years = int(np.round((end_dt - start_dt).total_seconds()/(60.*60.*24.*365.25),0))
    
    
    #before_idx, after_idx = split_on_time(dates_df, start_tstmp)
    #before_idx2, after_idx2 = split_on_time(dates_df, end_tstmp)
    
    # drop unnecessary columns
    if omit_columns is not None:
        df.drop(omit_columns,axis=1,inplace=True)
    
    
    # identify the set of training data that needs time adjustment -- i.e. if date is 
    indices_to_fix = dates_df[(dates_df.hire_tstmp< end_dt) & (dates_df.term_tstmp >= end_dt)].index
    # take the intersection of this and the training set
    idx_to_fix = list(set(indices_to_fix.tolist()).intersection(set(train_idx)))
    print "{0} indices need time adjustment".format(len(idx_to_fix))
    
    # create test and training sets.
    train_df = df.ix[train_idx].copy()
    test_df = df.ix[test_idx].copy()
    #holdout_df = df.ix[holdout_idx].copy()
    # calculate fraction in each class
    print len(train_df),len(test_df)
    print train_df[target_col].value_counts()
    x_class_train = train_df[target_col].value_counts().ix[1]/float(len(train_df))
    x_class_test = test_df[target_col].value_counts().ix[1]/float(len(test_df))
    print x_class_train, x_class_test
    #x_separated_holdout = holdout_df[target_col].value_counts().ix[1]/float(len(holdout_df))
    # correct status/sep_status, retired, etc. from training df
    characteristic_cols = ['status','sep_status','retired']
    train_df.loc[idx_to_fix,characteristic_cols]= 0 # reset to zero because term date is after split_tstmp
    # recalculate Age_years & Tenure_years
    ## now calculate age at termination date
    age_years = (end_dt - dates_df['birth_tstmp'])/np.timedelta64(1,'Y')
    tenure_years = (end_dt - dates_df['hire_tstmp'])/np.timedelta64(1,'Y')
    train_df['Age_years'] = age_years[train_idx]
    train_df['Tenure_years'] = tenure_years[train_idx]
    
    # correct time-sensitve columns --> concern is for recent job changes.
    """# assume today is '01-01-2015'
    today_tstmp = pd.to_datetime('2015-01-01')
    months_ago = (today_tstmp - split_tstmp)/np.timedelta64(1,'M')
    print months_ago
    for tcol in time_columns:
        print tcol
        if tcol == 'TELE_MOS':
    """        
    
    
    y_train = train_df[target_col].as_matrix().astype(tgt_type)
    #other_to_drop = characteristic_cols.remove(target_col[0])
    tdf = train_df.drop(characteristic_cols,axis=1).copy()
    feature_names = tdf.columns.tolist() # save the features_names
    X_train = tdf.as_matrix().astype(np.float)
    y_test = test_df[target_col].as_matrix().astype(tgt_type)
    tdf = test_df.drop(characteristic_cols,axis=1).copy()
    X_test = tdf.as_matrix().astype(np.float)
    frac_class = [x_class_train, x_class_test]
    
    return (X_train,y_train,X_test,y_test,feature_names,idx_to_fix,frac_class,range_in_years)

In [ ]:
create_time_slice(empl,empl_dates,'2007-01-01','2012-01-01','retired',temporal_cols)

In [ ]:
X,y,Xtest,ytest,feature_names,fix_idx,fc_vals,time_range_years = create_time_slice(empl,empl_dates,'2009-01-01','2012-01-01','status',temporal_cols)

In [ ]:
train_idx, test_idx = divide_by_start_end_time(empl_dates,date(2009,1,1),date(2012,1,1))

In [ ]:
empl.ix[train_idx][empl_dates.ix[train_idx].term_tstmp > date(2012,1,1)].status.value_counts()

In [ ]:
Xhh,yhh = create_accuracy_set(empl,fix_idx,'status')
#len(Xhh), len(X_holdout)

In [ ]:
empl_dates.ix[142113][['term_tstmp','hire_tstmp']]

In [ ]:
dftmp = empl.ix[train_idx][empl_dates.ix[train_idx].term_tstmp >= date(2012,1,1)].copy()
y_holdout = dftmp.status.as_matrix().astype(np.float)
dftmp.drop(['status','sep_status','retired'],inplace=True,axis=1)
X_holdout = dftmp.as_matrix().astype(np.float)
y_hh_pred = rfc_mdl.predict(X_holdout)
sns.heatmap(metrics.confusion_matrix(y_holdout,y_hh_pred), annot=True)
#plt.scatter(y_holdout,y_hh_pred)

In [ ]:
y_hh_proba = rfc_mdl.predict_proba(X_holdout)
plot_roc_curve(y_holdout,y_hh_proba)

In [ ]:
y_test_proba = rfc_mdl.predict_proba(Xtest)
plt.scatter(y_holdout,y_hh_proba[:,0],alpha=0.3,color='darkgreen')
plt.scatter(ytest+0.2,y_test_proba[:,0],alpha=0.3,color='dodgerblue')


In [ ]:
y_test_pred = rfc_mdl.predict(Xtest)
sns.heatmap(metrics.confusion_matrix(ytest,y_test_pred), annot=True)

In [ ]:
fc_vals

In [ ]:
single_rfc_500trees = ensemble.RandomForestClassifier(n_estimators=500,n_jobs=50)
rfc_mdl = single_rfc_500trees.fit(X,y)

In [ ]:
calculate_metrics(rfc_mdl,ytest,Xtest)

In [ ]:
sum(y_hh_pred)/float(len(y_hh_pred)), sum(y_holdout)/float(len(y_holdout))

In [ ]:
date(2015,1,1)

In [ ]:
empl_dates.term_tstmp.min()

In [ ]:
time_str_list = [str(a)+'-01-01' for a in xrange(1987,2015)]

In [ ]:
A = time_str_list[3::5]
for first, second in zip(A, A[1:]):
    print first, second

In [ ]:
len(time_str_list[3::5]), len(time_str_list[2::5])

#Begin with predicting retirements
* look at different time intervals
* shoot much more class imbalanced

In [ ]:
def build_time_slice_models(df,dates,times_list,tgt_col,time_columns,ntrees=500):
    measures =[]
    Xsets =[]
    ysets = []
    models =[]
    balance_fracs =[]
    
    for st_date,ed_date in zip(times_list,times_list[1:]):
        print st_date, ed_date
        X,y,Xtest,ytest,feature_names,fix_idx,fc_vals,duration = create_time_slice(df,dates,st_date,ed_date,tgt_col,time_columns)
        # check the set that we know the answer about
        Xknown,yknown = create_accuracy_set(empl,fix_idx,tgt_col)
        # construct a single RF
        single_rfc = ensemble.RandomForestClassifier(n_estimators=ntrees,n_jobs=50)
        rfc_mdl = single_rfc.fit(X,y) # fit on training data
        # append model to list of models
        models.append(rfc_mdl)
        # append datasets
        Xsets.append([X,Xtest,Xknown])
        ysets.append([y,ytest,yknown])
        plt.figure()
        m_acc_a,m_acc_m,m_acc_r = calculate_metrics(rfc_mdl,y_acc,X_acc)
        m_new_a,m_new_m,m_new_r = calculate_metrics(rfc_mdl,ytest,Xtest)
        measures.append([m_acc_a,m_acc_m, m_acc_r, m_new_a, m_new_m, m_new_r,st_date,ed_date,duration,len(y),len(ytest),len(yknown),fc_vals[0],fc_vals[1]])
    
    mdl_df = pd.DataFrame(data=measures,columns=['Acc_in','MCC_in','AUC_in','Acc_post','MCC_post','AUC_post',
                                                 'start_date','end_date','nyears','train_size','test_size','in_size',
                                                 'class_frac_train','class_frac_test'])
    #mdl_df_list.append(mdl_df)

    #dl_df = pd.DataFrame(mdl_df_list)
        #mdl_df.iloc[k,:] =measures
    return mdl_df,models,Xsets,ysets
        
        
        
        

In [ ]:
mm = build_time_slice_models(empl,empl_dates,A,'retired',temporal_cols)

In [ ]:
five_year_windows_df_list = []
five_year_windows_data = []
five_year_windows_models =[]
for i in xrange(0,5):
    print i, time_str_list[i::5]
    dd,mdl,xsets,ysets = build_time_slice_models(empl,empl_dates,time_str_list[i::5],'status',temporal_cols)
    five_year_windows_df_list.append(dd)
    five_year_windows_data.append([xsets,ysets])
    five_year_windows_models.append(mdl)
    

In [ ]:
m0 = build_time_slice_models(empl,empl_dates,time_str_list[::5],'retired',temporal_cols)

In [ ]:
len(five_year_windows_df_list)

In [ ]:
five_yr_df = pd.concat(five_year_windows_df_list,ignore_index=True)
five_yr_df.sort('start_date',inplace=True)

In [ ]:
five_yr_df.head()

In [ ]:
five_yr_df['MCC_post'].plot('o')
#.plot(kind='scatter',x='start_date',y='AUC_post')

In [ ]:
five_yr_df.plot(kind='scatter',x='AUC_in',y='AUC_post')

In [ ]:
ax = five_yr_df.plot(kind='scatter',y='AUC_in',x='class_frac_train')
five_yr_df.plot(kind='scatter',y='AUC_post',x='class_frac_test',color='deeppink',ax=ax)

In [ ]:
five_yr_df.plot(kind='scatter',x='AUC_post',y='test_size')

In [ ]:
pd.tools.plotting.scatter_matrix(five_yr_df[['Acc_in','AUC_in','Acc_post','AUC_post','class_frac_train','class_frac_test']], figsize=(10, 10), diagonal='kde')

## look at the Five year time slices Data Frame
* built on prediction of status

In [ ]:
five_yr_df.sort('AUC_post',ascending=False)

## what are the features ?

In [ ]:
five_yr_win_X = []
five_yr_win_Y = []
five_yr_win_mdl = []
for a in xrange(0,len(five_year_windows_data)):
    five_yr_win_X+=[x for x in five_year_windows_data[a][0]]
    five_yr_win_Y+=[x for x in five_year_windows_data[a][1]]
    five_yr_win_mdl+=[x for x in five_year_windows_models[a]]
    

In [ ]:
feature_names = empl.columns.tolist()
for xname in ['status','sep_status','retired']:
    feature_names.remove(xname)

In [ ]:
five_yr_win_feature_importances = []
for i,mdl in enumerate(five_yr_win_mdl):
    five_yr_win_feature_importances.append(plotFI(mdl,feature_names))
    

In [ ]:
len(five_yr_win_feature_importances)

In [ ]:
five_yr_win_feature_importances[15].sort('value',ascending=False)

In [ ]:
# what is the ratio of the test size to the train size?
five_yr_df['tt_ratio'] = five_yr_df['test_size']/five_yr_df['train_size']
five_yr_df.tt_ratio

In [ ]:
plt.scatter(five_yr_df['class_frac_train'],five_yr_df['AUC_post'],s=10/five_yr_df['tt_ratio'])
plt.xlabel('Fraction of Train Set Separating')
plt.ylabel('AUC on test set')

In [ ]:
five_yr_df[five_yr_df.AUC_post > 0.7].head()

In [ ]:
for my_idx in [8,3,13,21,17]:
    
    plt.figure()
    my_date = five_yr_df.loc[my_idx,'start_date']
    plt.title(my_date)
    five_yr_win_feature_importances[my_idx].sort('value',ascending=False)['value'].head(10).plot(kind='barh')

### repeat for 3 year intervals

In [ ]:
three_year_windows_df_list = []
three_year_windows_data = []
three_year_windows_models =[]
# generate the datasets, models and begin analysis.
for i in xrange(0,3):
    print i, time_str_list[i::3]
    dd,mdl,xsets,ysets = build_time_slice_models(empl,empl_dates,time_str_list[i::3],'status',temporal_cols)
    three_year_windows_df_list.append(dd)
    three_year_windows_data.append([xsets,ysets])
    three_year_windows_models.append(mdl)
    
# join and sort the dataFrame
three_yr_df = pd.concat(three_year_windows_df_list,ignore_index=True)
three_yr_df.sort('start_date',inplace=True)
# add colum for test-train ratio:
three_yr_df['tt_ratio'] = three_yr_df['test_size']/three_yr_df['train_size']

# now flatten output
three_yr_win_X = []
three_yr_win_Y = []
three_yr_win_mdl = []
for a in xrange(0,len(three_year_windows_data)):
    three_yr_win_X+=[x for x in three_year_windows_data[a][0]]
    three_yr_win_Y+=[x for x in three_year_windows_data[a][1]]
    three_yr_win_mdl+=[x for x in three_year_windows_models[a]]

# obtain the feature importances    
three_yr_win_feature_importances = []
for i,mdl in enumerate(three_yr_win_mdl):
    three_yr_win_feature_importances.append(plotFI(mdl,feature_names))

In [ ]:
plt.scatter(three_yr_df['class_frac_train'],three_yr_df['AUC_post'],s=10/three_yr_df['tt_ratio'],color='darkgoldenrod')
plt.xlabel('Training Separated Fraction')
plt.ylabel('Test AUC') 

In [ ]:
# get top AUC results
topAUCresults = three_yr_df[three_yr_df.AUC_post > 0.7].sort('AUC_post',ascending=False)
top_idx_list = topAUCresults.index.tolist()

In [ ]:
for my_idx in top_idx_list:    
    plt.figure()
    my_date = three_yr_df.loc[my_idx,'start_date']
    my_auc = three_yr_df.loc[my_idx,'AUC_post']
    plt.title('Start date: '+my_date+'  AUC= '+str(my_auc))
    three_yr_win_feature_importances[my_idx].sort('value',ascending=False)['value'].head(10).plot(kind='barh',color='darkgoldenrod')

In [ ]:
len(three_yr_win_Y[3][1]), three_yr_df.ix[3]['test_size']

## look at the Confusion matrices of some of these 'top results'

In [ ]:
for my_idx in top_idx_list:    
    plt.figure()
    my_date = three_yr_df.loc[my_idx,'start_date']
    my_auc = three_yr_df.loc[my_idx,'AUC_post']
    plt.title('Confusion matrix')
    plt.suptitle('Start date: '+my_date+'  AUC= '+str(my_auc))
    sns.heatmap(metrics.confusion_matrix(three_yr_win_Y[my_idx][1],three_yr_win_mdl[my_idx].predict(three_yr_win_X[my_idx][1])),annot=True,fmt="d")
    


## In a general sense these models are UNDER predicting Separations ('1')
* try to rescale the threshold using known results??

In [ ]:
plt.scatter(three_yr_df['AUC_in'],three_yr_df['AUC_post'])
plt.xlabel('AUC_vetted')
plt.ylabel('AUC_test')
plt.title('3 year windows')

In [ ]:
iix = 22
my_mdl = three_yr_win_mdl[iix]
plot_roc_curve(three_yr_win_Y[iix][2],my_mdl.predict_proba(three_yr_win_X[iix][2]))
plot_roc_curve(three_yr_win_Y[iix][1],my_mdl.predict_proba(three_yr_win_X[iix][1]))
#plt.title('Within set'

In [ ]:
sns.heatmap(metrics.confusion_matrix(three_yr_win_Y[iix][2],three_yr_win_mdl[iix].predict(three_yr_win_X[iix][2])),annot=True,fmt="d")

In [ ]:
plt.figure()
plt.plot(three_yr_win_Y[iix][2],'o')
#plt.figure()
plt.plot(three_yr_win_mdl[iix].predict_proba(three_yr_win_X[iix][2])[:,0],'o',color='midnightblue',alpha=0.3)
plt.plot(three_yr_win_mdl[iix].predict_log_proba(three_yr_win_X[iix][2])[:,0],'o',color='skyblue',alpha=0.3)
plt.plot(three_yr_win_mdl[iix].predict(three_yr_win_X[iix][2]),'*',color='indianred',alpha=0.3)

In [ ]:
my_pred_prob0 = three_yr_win_mdl[iix].predict_proba(three_yr_win_X[iix][2])[:,0]
my_pred_logprob0 = three_yr_win_mdl[iix].predict_log_proba(three_yr_win_X[iix][2])[:,0]
plt.hist(my_pred_prob0,range=[0,1],bins=50,color='darkorchid')

In [ ]:
np.round(1-my_pred_prob0,0).sum()

In [ ]:
my_tru = three_yr_win_Y[iix][2]
shift_pred = 1-my_pred_prob0 + 0.48#,np.round(x,0))
#sns.heatmap(metrics.confusion_matrix(my_tru,shfit_pred),annot=True,fmt="d")
np.round(shift_pred,0).sum()

In [ ]:
sns.heatmap(metrics.confusion_matrix(my_tru,np.round(shift_pred,0)),annot=True,fmt="d")

In [ ]:
metrics.matthews_corrcoef(my_tru,np.round(shift_pred,0)),three_yr_df.ix[iix]['MCC_in']

In [ ]:
my_shift_pred = 1-(1-three_yr_win_mdl[iix].predict_proba(three_yr_win_X[iix][2]) + 0.48)
plot_roc_curve(my_tru,my_shift_pred)#,np.round(x,0))plot_roc_curve(my_tru,shift_pred)
plot_roc_curve(my_tru,three_yr_win_mdl[iix].predict_proba(three_yr_win_X[iix][2]))

In [ ]:
my_test_y = three_yr_win_Y[iix][1]
my_test_y_proba = my_mdl.predict_proba(three_yr_win_X[iix][1])
my_test_y_pred = my_mdl.predict(three_yr_win_X[iix][1])
my_shift_pred_int = np.round(1-my_test_y_proba[:,0]+0.48,0)
metrics.matthews_corrcoef(my_test_y,my_shift_pred_int), three_yr_df.ix[iix]['MCC_post']

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(121)
sns.heatmap(metrics.confusion_matrix(my_test_y,my_shift_pred_int),annot=True,fmt="d")
plt.title('After making shift')
plt.subplot(122)
sns.heatmap(metrics.confusion_matrix(my_test_y,my_test_y_pred),annot=True,fmt="d")

In [ ]:
three_yr_df.sort('MCC_post',ascending=False)

#Aside predict retirements
* same procedure of taking a range of time and holding out a sample

In [ ]:
RET_measures = []
RET_dsets = []
RET_models = []
for tslice in time_slices_range:
    print tslice
    X,y,Xtest,ytest,feature_names,fix_idx = create_time_split(empl,empl_dates,tslice,'retired',temporal_cols)
    #extract accuracy set
    X_acc,y_acc = create_accuracy_set(empl,fix_idx,'retired')
    # do a single RF
    single_rfc_500trees = ensemble.RandomForestClassifier(n_estimators=500,n_jobs=50)
    rfc_mdl = single_rfc_500trees.fit(X,y) # fit on training data
    RET_models.append(rfc_mdl)
    RET_dsets.append([X,Xtest,X_acc,y,ytest,y_acc])
    m_acc_a,m_acc_m,m_acc_r = calculate_metrics(rfc_mdl,y_acc,X_acc)
    m_new_a,m_new_m,m_new_r = calculate_metrics(rfc_mdl,ytest,Xtest)
    RET_measures.append([m_acc_a,m_acc_m, m_acc_r, m_new_a, m_new_m, m_new_r])
    
    
# push the data/parameters into a small df
RETrfmodels_df = pd.DataFrame(data=RET_measures,index=time_slices_range,columns=['Acc_in','MCC_in','AUC_in','Acc_post','MCC_post','AUC_post'])
RETrfmodels_df.head()

# transform to get the dataset sizes
RETds_size = []
for b in xrange(0,len(RET_dsets)):
    RETds_size.append([len(RET_dsets[b][a]) for a in xrange(0,len(RET_dsets))])

RETds_size=np.array(RETds_size) # convert to an array
# add to the dataframe
RETrfmodels_df['Train_size'] = RETds_size[:,0]
RETrfmodels_df['in_size' ] = RETds_size[:,1]
RETrfmodels_df['post_size'] = RETds_size[:,2]

In [ ]:
RETrfmodels_df

## look at some jobcodes 
* 6200 == CRC CallRep
* 1876, 1877 == QB Rep

In [ ]:
sum(emplfull.JOBCODE==6200)

In [ ]:
emplfull[emplfull.JOBCODE==6200]['JOB_FAMILY'].unique()

In [ ]:
sum(emplfull.JOB_FAMILY=='QBSREP')

In [ ]:
sum(emplfull.JOB_FAMILY=='CLMREP')

In [ ]:
len(emplfull.JOB_FAMILY.unique()),len(empl.job_fcode.unique())


In [ ]:
len(emplfull.JOBCODE.unique())

In [ ]:
emplfull[emplfull.JOBCODE==1876]['JOB_FAMILY'].unique()

In [ ]:
empl.columns[-10:]

In [ ]:
rscrep_idx = emplfull[emplfull.JOB_FAMILY=='RSCREP'].index
empl.ix[rscrep_idx].grade_code.unique(), empl.ix[rscrep_idx].job_fcode.unique()

# Survival Analysis (again)

In [ ]:
from lifelines import KaplanMeierFitter

## baseline KM Fitter

In [ ]:
kmf0 = KaplanMeierFitter()
plt.figure(figsize=(14,6))
kmf0.fit(emplfull.Tenure_months,event_observed=emplfull.status)
kmf0.plot(color='darkslategray')
plt.ylabel('Survival Distribution Function (Separation)')
plt.xlabel('Tenure months')
plt.title('KaplanMeier Survival: ALL employees')

In [ ]:
kmf0 = KaplanMeierFitter()
plt.figure(figsize=(14,6))
kmf0.fit(emplfull.Tenure_months,event_observed=emplfull.status)
kmf0.plot(color='darkslategray')
plt.ylabel('Survival Distribution Function (Separation)')
plt.xlabel('Tenure months')
plt.title('KaplanMeier Survival: ALL employees')
plt.xlim(0,96)

In [ ]:
job_family_value_counts = emplfull.JOB_FAMILY.value_counts()
job_family_value_counts[job_family_value_counts > 400]

In [ ]:
# work with those Job_families that have more than 700 members
job_family_value_counts[job_family_value_counts > 700].sum(), len(job_family_value_counts[job_family_value_counts > 700])

In [ ]:
# create groups
jfam_grps = job_family_value_counts.iloc[:23].index.tolist()


In [ ]:
jfam_index_dict = {}
for family in jfam_grps:
    jfam_index_dict[family] = emplfull[emplfull.JOB_FAMILY == family].index.tolist()
print len(jfam_index_dict)

In [ ]:
emplfull.Tenure_months.describe()

In [ ]:
#tenure_month_range = [0,600]
tenure_month_range = np.linspace(0,600,601)

In [ ]:
col_list = ['darkmagenta','midnightblue','darkolivegreen','darkgreen','darkred','aquamarine',
            'deeppink','darkorange','darksalmon','darkturquoise','darkkhaki','lime','darkgoldenrod']

In [ ]:
jfam_grps[1], len(jfam_index_dict[jfam_grps[1]])

In [ ]:
fig = plt.figure(figsize=(12,16))
ax1 = fig.add_subplot(411)
ax2 = fig.add_subplot(412)
ax3 = fig.add_subplot(413)
ax4 = fig.add_subplot(414)
kmf_jf0 = KaplanMeierFitter()
kmf_jf0.fit(emplfull.Tenure_months, timeline=tenure_month_range, event_observed=emplfull.status,label='All')
jfcn_sf_df = pd.DataFrame(kmf_jf0.survival_function_)
#ax = fig.add_subplot(111)
kmf_jf0.plot(ax=ax1,c='darkslategray')
kmf_jf0.plot(ax=ax2,c='darkslategray')
kmf_jf0.plot(ax=ax3,c='darkslategray')
kmf_jf0.plot(ax=ax4,c='darkslategray')
for jfx in xrange(0,23):#[1,2,3,4,5,6]:#jfx = 1
    jf = jfam_grps[jfx]
    jfid = jfam_index_dict[jf]
    print jfx,jf,len(jfid)
    kmf_jf0.fit(emplfull.Tenure_months.ix[jfid], timeline=tenure_month_range, event_observed=emplfull.status.ix[jfid],label=jf)
#kmf_mgr1.fit(vc_empl4[~mgrA].Tenure_months, timeline=tenure_month_range,event_observed=vc_empl4[~mgrA].status,label='non-')
    if jfx % 4 == 0:
        ax = ax1
    elif jfx % 4 == 1:
        ax = ax2
    elif jfx % 4 == 2:
        ax = ax3
    else:
        ax= ax4
    cname = col_list[jfx % len(col_list)]
    #if jfx > len(col_list):
    #    cname = col_list[jfx % 7]
    kmf_jf0.plot(ax=ax,color=cname)
    jfcn_sf_df = pd.concat([jfcn_sf_df,kmf_jf0.survival_function_],axis=1)

plt.ylim(0,1.05)
plt.ylabel('Survival Distribution Function')
plt.xlabel('Tenure Months')

In [ ]:
#Repeat looking only at <= 8 years == 96 months
fig = plt.figure(figsize=(12,16))
ax1 = fig.add_subplot(411)
ax2 = fig.add_subplot(412)
ax3 = fig.add_subplot(413)
ax4 = fig.add_subplot(414)
subplot_axis_list = [ax1, ax2, ax3, ax4]
kmf_jf0 = KaplanMeierFitter()
kmf_jf0.fit(emplfull.Tenure_months, timeline=tenure_month_range, event_observed=emplfull.status,label='All')
jfcn_sf_df = pd.DataFrame(kmf_jf0.survival_function_)
#ax = fig.add_subplot(111)
kmf_jf0.plot(ax=ax1,c='darkslategray')
kmf_jf0.plot(ax=ax2,c='darkslategray')
kmf_jf0.plot(ax=ax3,c='darkslategray')
kmf_jf0.plot(ax=ax4,c='darkslategray')
for jfx in xrange(0,23):#[1,2,3,4,5,6]:#jfx = 1
    jf = jfam_grps[jfx]
    jfid = jfam_index_dict[jf]
    print jfx,jf,len(jfid)
    kmf_jf0.fit(emplfull.Tenure_months.ix[jfid], timeline=tenure_month_range, event_observed=emplfull.status.ix[jfid],label=jf)
#kmf_mgr1.fit(vc_empl4[~mgrA].Tenure_months, timeline=tenure_month_range,event_observed=vc_empl4[~mgrA].status,label='non-')
    if jfx % 4 == 0:
        ax = ax1
    elif jfx % 4 == 1:
        ax = ax2
    elif jfx % 4 == 2:
        ax = ax3
    else:
        ax= ax4
    cname = col_list[jfx % len(col_list)]
    #if jfx > len(col_list):
    #    cname = col_list[jfx % 7]
    kmf_jf0.plot(ax=ax,color=cname)
    jfcn_sf_df = pd.concat([jfcn_sf_df,kmf_jf0.survival_function_],axis=1)
[my_ax.set_xlim([0,96]) for my_ax in subplot_axis_list]
plt.ylim(0,1.05)
#plt.xlim(0,96)
plt.ylabel('Survival Distribution Function')
plt.xlabel('Tenure Months')

In [ ]:
jfcn_sf_df.head()

## How about hazards?

In [ ]:
from lifelines import CoxPHFitter

In [ ]:
cf_tenure = CoxPHFitter()


In [ ]:
myCols = ['HAVE_INS','SEX','Age_years','job_fcode','Tenure_years','status']

In [ ]:
%%time
cf_tenure.fit(empl[myCols],'Tenure_years',event_col ='status')
cf_tenure.hazards_

In [ ]:
np.exp(cf_tenure.hazards_)

In [ ]:
cf_tenure.summary()

In [ ]:
cf_tenure.baseline_cumulative_hazard_.ix[:6].diff().plot()

In [ ]:
cf_tenure.confidence_intervals_

In [ ]:
cf_tenure.baseline_hazard_.ix[:6].plot()
##plt.ylim(0,1)
#plt.xlim(0,7)

### so this is fine  -- except that job_fcode is label encoded but not in a meaningful (i.e. ordinal way)
* Better to treat it as categorical. 
* 1456 categories seems like a lot. Lookint at KMF above for top 23 categories each contain more than 7000 members and account for 72% of the observations.
* create a function to group these better?!
    - there are 272 job_families that have only 1 member.

In [ ]:
print " there are {0} job_fcode (JOB_FAMILYs) that have only 1 element".format(sum(empl.job_fcode.value_counts() == 1))


In [ ]:
print " there are {0} job_fcode (JOB_FAMILYs) that have only 2 elements".format(sum(empl.job_fcode.value_counts() == 2))
print " there are {0} job_fcode (JOB_FAMILYs) that have only 3 elements".format(sum(empl.job_fcode.value_counts() == 3))


In [ ]:
emplfull.JOB_FAMILY.value_counts().tail()

In [ ]:
emplfull[emplfull.JOB_FAMILY == 'FSCCRD'][['GRADE', 'JOBCODE']]

In [ ]:
emplfull[emplfull.JOBCODE == 90][['GRADE','JOB_FAMILY','status','TERMINATION_DT','JOBCODE']]

In [ ]:
emplfull[emplfull.JOBCODE == 36].GRADE.value_counts()

In [ ]:
 emplfull.JOBCODE.value_counts()

In [ ]:
import patsy
Xll = patsy.dmatrix('Age_years + SEX + C(job_fcode) +HAVE_INS + SAL1', empl, return_type='dataframe')

In [ ]:
print Xll.shape
Xll['T' ] =empl.Tenure_years
Xll['E'] = empl.status

### Look at which categorical job_fcodes have more than 500 members 

In [ ]:
cat_job_fcode_to_keep = [x for x in Xll.columns if x.startswith('C(job') and sum(Xll[x]) >= 500]

In [ ]:
col_to_keep =['Age_years','SEX','HAVE_INS','SAL1','T','E']
col_to_keep += cat_job_fcode_to_keep
len(col_to_keep)

In [ ]:
%%time 
cf = CoxPHFitter()
cf.fit(Xll[col_to_keep],'T',event_col ='E')

In [ ]:
print cf.summary()

In [ ]:
emplfull.JOB_FAMILY.value_counts()

In [ ]:
### plot the cumulative hazards
#cf.baseline_survival_.ix[:8].plot()
cf.baseline_cumulative_hazard_.ix[:8].plot()

In [ ]:
expbeta = np.exp(cf.hazards_).T
expbeta.sort('coef',ascending=False)

In [ ]:
#C(job_fcode)[T.885]	1.243266
#C(job_fcode)[T.1108]	1.172821
#C(job_fcode)[T.327]	1.139246
#C(job_fcode)[T.1061]	1.124839
#C(job_fcode)[T.796]
top_hazard_job_fcode = [885,1108,327,1061,796]

for code in top_hazard_job_fcode:
    jfcd_cat = 'C(job_fcode)[T.'+str(code)+']'
    expHazard = expbeta.ix[jfcd_cat].values[0]
    case1 = empl[empl['job_fcode']==code].index[0]
    print code, len(empl[empl['job_fcode']==code]), emplfull.ix[case1].JOB_FAMILY,expHazard

In [ ]:
expbeta.ix[jfcd_cat].values[0]

In [ ]:
# function to group singletons of a categorical object
def truncate_categorical(df,column,min_thresh,group_singletons=True):
    vc = df.column.value_counts()
    

In [ ]:
from lifelines.utils import k_fold_cross_validation

cf = CoxPHFitter()
#empl[myCols],'Tenure_years',event_col ='status')
scores = k_fold_cross_validation(cf, empl[myCols], 'Tenure_years', event_col='status', k=3)
print scores
print scores.mean()
print scores.std()


In [ ]:
from lifelines import AalenAdditiveFitter

In [ ]:
aaf_tenure = AalenAdditiveFitter(penalizer=1.0, fit_intercept = True)
aaf_tenure.fit(Xll, 'T','E')

In [ ]:
aaf_tenure.hazards_['Age_years'].ix[:8].plot()

In [ ]:
aaf_tenure.hazards_['job_fcode'].ix[:5].plot()

In [ ]:
#aaf_tenure.plot(columns=['baseline','Age_years','INTERN','SEX','ANNUAL_RT'],ix=slice(1,15))

In [ ]:
aaf_tenure.hazards_.head()

In [ ]:
aaf_tenure.smoothed_hazards_()[['job_fcode','SAL1']].ix[:5].plot()

In [ ]:
kmf_jfothr = KaplanMeierFitter()
fnm = 'BOMISD'
kmf_jfothr.fit(emplfull[emplfull['JOB_FAMILY']==fnm].Tenure_months, timeline=tenure_month_range, event_observed=emplfull[emplfull['JOB_FAMILY']=='SAKBSY'].status,label=fnm)
kmf_jfothr.plot()

In [ ]:
%matplotlib inline

## Begin prediction of tenure year

### try RF regression

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn import cross_validation

In [ ]:
y = empl3['Tenure_years'].values.astype(np.float)
df = empl3.drop(['Tenure_years','status','sep_status','retired'],axis=1).copy()
X = df.as_matrix().astype(np.float)
y.shape, X.shape

In [ ]:
#y_tenure_class = pd.cut(empl3.Tenure_years,np.arange(0,empl3.Tenure_years.max()+1,1))
y_tenure_class = pd.cut(empl2.Tenure_years,[0,1,3,5,10,15,25,200],labels=False,right=False)#,labels=[1,2,3,4,5,6,7,8])#.values
#y_tenure_class

In [ ]:
pd.value_counts(y_tenure_class).plot(kind='bar')#.reindex(y_tenure_class.levels)

### tenure_years classes = {0:[0,1),1:[1,3),2:[3,5),3:[5,10),4:[10,15),5:[15,25),6:[25,end)}

In [ ]:
%%time
print('RF Classifier: {:.2f}'.format(metrics.accuracy_score(y_tenure_class, kfold_cv(X, y_tenure_class, ensemble.RandomForestClassifier))))

In [ ]:
rfc_conf_matrix = metrics.confusion_matrix(y_tenure_class, kfold_cv(X, y_tenure_class, ensemble.RandomForestClassifier))

In [ ]:
sns.heatmap(rfc_conf_matrix, annot=False)

In [ ]:
rfc_conf_matrix

In [ ]:
%%time 
forest = ensemble.ExtraTreesClassifier(n_jobs=20)
forest.fit(X,y_tenure_class)
importances= forest.feature_importances_

In [ ]:
forest_std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
indices = np.argsort(importances)[::-1]


In [ ]:
feature_names = df.columns[indices].tolist()

In [ ]:
#for f in range(20):
nfeat = 30
plt.barh(range(nfeat),importances[indices[:nfeat]],yerr=forest_std[indices[:nfeat]],align="center")
plt.yticks(range(nfeat),feature_names[:nfeat])


## Now take this model and predict into the future:
* first define set of currently employeed (not-separated) employees
* augment time_sensitive measures

In [ ]:
empl_active = empl3[empl3.status == 0].copy()
empl_active.shape

In [ ]:
empl3[temporal_cols[:-3]].apply(lambda x: x+12.0)

In [ ]:
df = empl3.drop(['Tenure_years','status','sep_status','retired'],axis=1).copy()
X = df.as_matrix().astype(np.float)
y.shape, X.shape

In [ ]:
active_tenure_class = y_tenure_class[empl_active.index]

In [ ]:
empl_active[empl_active.REMOTE==0].TELE_MOS.values.min()


In [ ]:
def prep_for_prediction(df,time_columns,delta_time=0.0,cols_to_drop = ['Tenure_years','status','sep_status','retired']):
    monthly_cols = [x for x in time_columns if x.endswith('MOS')]
    d2 = df.copy()
    d2[monthly_cols[:-1]]= df[monthly_cols[:-1]].apply(lambda x: x+12.0*delta_time)
    # special column is TELE_MOS
    d2[d2.REMOTE==1][monthly_cols[-1]] = df[df.REMOTE==1][monthly_cols[-1]].apply(lambda x:x+delta_time*12.0)
    
    annual_cols = [x for x in time_columns if x.endswith('years')]
    d2[annual_cols]=df[annual_cols].apply(lambda x: x+1.0*delta_time)
    d2.drop(cols_to_drop,axis=1,inplace=True)
    X = d2.as_matrix().astype(np.float)
    print X.shape
    return X,df.Tenure_years.values+1.0*delta_time

In [ ]:
myX,myY = prep_for_prediction(empl_active,temporal_cols,delta_time=0)

In [ ]:
myY[:3]

In [ ]:
active_tenure_class

In [ ]:
forest.predict(myX)

In [ ]:
forest.predict_proba(myX)

In [ ]:
empl_active['Tenure_years'].head()

## May 12, 2015 
### Attempt RandomForest Regressor (of Tenure Years)
1. use all data --> simple test-train split

In [ ]:

test_fraction = 0.3
X_train, X_test , y_train, y_test = cross_validation.train_test_split(X,y,test_size=test_fraction)

In [ ]:
%%time 
RFR = ensemble.RandomForestRegressor(n_estimators=200,max_features="sqrt",n_jobs=30)
RFR.fit(X_train,y_train)
RFR_importances= RFR.feature_importances_

In [ ]:
RFR.score(X_test,y_test)

In [ ]:
RFR_importances_std = np.std([tree.feature_importances_ for tree in RFR.estimators_], axis=0)
RFR_indices = np.argsort(RFR_importances)[::-1]
RFR_feature_names = df.columns[RFR_indices].tolist()

In [ ]:
#for f in range(20):
nfeat = 30
plt.barh(range(nfeat),RFR_importances[RFR_indices[:nfeat]],yerr=RFR_importances_std[RFR_indices[:nfeat]],color='darkturquoise')#align="center")
plt.yticks(range(nfeat),RFR_feature_names[:nfeat])


In [ ]:
def obtain_feature_importances(classifier,plot_flag=True):
    

### make a scatter plot of the prediction and actual

In [ ]:
y_prediction = RFR.predict(X)
plt.scatter(y,y_prediction,alpha=0.3)
plt.xlabel('True Tenure (years)')
plt.ylabel('Predicted Tenure (years)')
x_line_val = np.arange(0,60)
plt.plot(x_line_val,x_line_val,lw=3,color='k')

In [ ]:
#zoom in on <=10 years
y_prediction = RFR.predict(X)
plt.scatter(y,y_prediction,alpha=0.3)
plt.xlabel('True Tenure (years)')
plt.ylabel('Predicted Tenure (years)')
x_line_val = np.arange(0,11)
plt.plot(x_line_val,x_line_val,lw=3,color='k')
plt.xlim(0,10)
plt.ylim(0,10)

In [ ]:
# look at current employees.
np.shape(y[empl_active.index]), np.shape(X[empl_active.index,:])

In [ ]:
y_active_pred = RFR.predict(X[empl_active.index,:])
plt.scatter(y[empl_active.index],y_active_pred,alpha=0.3,color='chocolate')
plt.xlabel('True Tenure (years)')
plt.ylabel('Predicted Tenure (years)')
x_line_val = np.arange(0,11)
plt.plot(x_line_val,x_line_val,lw=3,color='k')
plt.xlim(0,10)
plt.ylim(0,10)

In [ ]:
def prep_for_prediction(df,time_columns,delta_time=0.0,cols_to_drop = ['Tenure_years','status','sep_status','retired']):
    monthly_cols = [x for x in time_columns if x.endswith('MOS')]
    d2 = df.copy()
    d2[monthly_cols[:-1]]= df[monthly_cols[:-1]].apply(lambda x: x+12.0*delta_time)
    # special column is TELE_MOS
    d2[d2.REMOTE==1][monthly_cols[-1]] = df[df.REMOTE==1][monthly_cols[-1]].apply(lambda x:x+delta_time*12.0)
    
    annual_cols = [x for x in time_columns if x.endswith('years')]
    d2[annual_cols]=df[annual_cols].apply(lambda x: x+1.0*delta_time)
    d2.drop(cols_to_drop,axis=1,inplace=True)
    X = d2.as_matrix().astype(np.float)
    print X.shape
    return X,df.Tenure_years.values+1.0*delta_time

In [ ]:
RFR.score(X[empl_active.index,:],y[empl_active.index])

## what about applying this model to a future workforce?

In [ ]:
def create_future_workforce(df,time_columns,delta_time=0.0,cols_to_drop = ['Tenure_years','status','sep_status','retired']):
    # deal with monthly columns
    monthly_cols = [x for x in time_columns if x.endswith('MOS')]
    d2 = df.copy()
    d2[monthly_cols[:-1]]= df[monthly_cols[:-1]].apply(lambda x: x+12.0*delta_time)
    # special column is TELE_MOS
    d2.loc[d2.REMOTE==1,monthly_cols[-1]] = df[df.REMOTE==1][monthly_cols[-1]].apply(lambda x:x+delta_time*12.0)
    #deal with annual columns
    annual_cols = [x for x in time_columns if x.endswith('years')]
    d2[annual_cols]=df[annual_cols].apply(lambda x: x+1.0*delta_time)
    d2.drop(cols_to_drop,axis=1,inplace=True)
    X = d2.as_matrix().astype(np.float)
    print X.shape
    return X,df.Tenure_years.values+1.0*delta_time

In [ ]:
myX,future_y = create_future_workforce(empl_active,temporal_cols,delta_time=1.0)

In [ ]:
future_y[:4], y[empl_active.index[:4]]

In [ ]:
y_active_pred = RFR.predict(X[empl_active.index,:])
plt.scatter(y[empl_active.index],y_active_pred,alpha=0.3,color='lightgreen')
plt.xlabel('True Tenure (years)')
plt.ylabel('Predicted Tenure (years)')
x_line_val = np.arange(0,61)
plt.plot(x_line_val,x_line_val,lw=3,color='k')
#plt.xlim(0,10)
#plt.ylim(0,10)

In [ ]:
y_active_pred_one = RFR.predict(myX)
plt.scatter(future_y,y_active_pred_one,alpha=0.3,color='chocolate')
plt.xlabel('True Tenure (years)')
plt.ylabel('Predicted Tenure (years)')
x_line_val = np.arange(0,61)
plt.plot(x_line_val,x_line_val,lw=3,color='k')
#plt.xlim(0,10) 
#plt.ylim(0,10)
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
y_temp = (np.vstack([future_y, np.ones(len(future_y))])).T
#np.shape(y_temp)
lm.fit(y_temp,y_active_pred_one)

# print intercept and coefficients
print lm.intercept_
print lm.coef_
xx = np.linspace(0,60,120)
yy = lm.intercept_+lm.coef_[0]*xx
plt.plot(xx,yy,lw=2,color='mediumvioletred')
r2 = metrics.r2_score(future_y,y_active_pred_one)
mse = np.mean((y_active_pred_one- future_y)**2)
print " R^2 is {0} and MSE is {1}".format(r2, mse)
# plot an interval around this of 1 year
ci_alpha = 0.95


In [ ]:
np.shape(y_active_pred_one), np.shape(future_y)

In [ ]:
import statsmodels.api as sm

model = sm.OLS(y_active_pred_one,future_y)
results = model.fit()
print " Parameters are: "
print results.params

In [ ]:
pred_one_mat = np.array(y_active_pred_one)
np.shape(pred_one_mat)

In [ ]:
lm.fit(y_temp
       ,future_y)

In [ ]:
plt.hist(future_y - y_active_pred_one,color='crimson',bins=30)
plt.xlabel('Difference between future tenure and predicted tenure')
plt.ylabel('Counts')

### compare to ExtraTreesRegressor and Gradient Boosted Trees

In [ ]:
## so next question is whether or not there are differences in terms of the individuals 

## Alternative approach:
### Simply classify as separated or not --> status is target variable
* use reduced dataset
* whiten the data

In [ ]:
y = empl3['status'].values.astype(np.float)
df = empl3.drop(['sep_status','retired','status'],axis=1).copy()
X = df.as_matrix().astype(np.float)
y.shape, X.shape

In [ ]:
"Fraction of separated: {0}".format(sum(y)/float(len(y)) )

In [ ]:
# preprocess this data set (whiten it)
X = preprocessing.StandardScaler().fit_transform(X)


In [ ]:
from sklearn import tree
from sklearn import svm
from sklearn import ensemble
from sklearn import neighbors
from sklearn import linear_model
from sklearn import metrics

In [ ]:
## define my classifiers
classifiers_list = [
    linear_model.PassiveAggressiveClassifier,
    linear_model.LogisticRegression,
    neighbors.KNeighborsClassifier,
    svm.SVC,
    tree.DecisionTreeClassifier,
    ensemble.RandomForestClassifier,
    ensemble.GradientBoostingClassifier]

#neighbors.KNeighborsClassifier(n_neighbors=3),
#    svm.SVC(kernel="linear", C=0.025),
#    svm.SVC(gamma=2, C=1),
#    tree.DecisionTreeClassifier(max_depth=10,max_features='auto'),
#    ensemble.RandomForestClassifier(max_depth=10, n_estimators=500, max_features='auto',n_jobs=30),
#    ensemble.GradientBoostingClassifier(),
    #AdaBoostClassifier(),
    #linear_model.LogisticRegression()]
    #GaussianNB(),
    #LDA(),
    #QDA()]

In [ ]:
%%time
y_pred = kfold_cv(X,y,classifiers_list[-2])


In [ ]:
auc_score = metrics.accuracy_score(y,y_pred)
print auc_score, classifiers_list[-2]

In [ ]:
print('Dump Classifier: {:.2f}'.format(metrics.accuracy_score(y, [0 for ii in y.tolist()])))

In [ ]:
rfc_conf_matrix_tenure = metrics.confusion_matrix(y,y_pred)
sns.heatmap(rfc_conf_matrix_tenure, annot=True,  fmt='')
plt.title('Random Forest Classifier Confusion Matrix')

## okay so what are the features responsible for this??

In [ ]:
np.shape(y_pred)

In [ ]:
def kfold_cv2(X, y, clf_class, shuffle=True, n_folds=10, **kwargs):
    k_fold = cross_validation.KFold(len(y), n_folds=n_folds, shuffle=shuffle)
    #y_pred = y.copy()
    kf_fits = []
    for ii, jj in k_fold:
        X_train, X_test = X[ii], X[jj]
        y_train = y[ii]
        clf = clf_class(**kwargs)
        clf.fit(X_train,y_train)
        kf_fits.append(clf)
        #y_pred[jj] = clf.predict(X_test)
    return k_fold,kf_fits

In [ ]:
kf_rfc,kf_fits = kfold_cv2(X,y,classifiers_list[-2])
print len(kf_fits)

In [ ]:
kf_fits[0].get_params

In [ ]:
# function to plot feature_importances for RF
def plotFI(forest,featureNames=[]):
    featureImportances=forest.feature_importances_
    # sort the importances from biggest to least
    indices = np.argsort(featureImportances)[::-1]
    estimators = forest.estimators_
    # calculate the variance over the forest 
    
    std = np.std([tree.feature_importances_ for tree in estimators],axis=0)
    # print summary statement
    nfeatures = len(featureImportances)
    print("Number of Features: %d" % (nfeatures))
    print("Number of Trees: %d" %(len(estimators)))
    
    #print featureNames
    if len(featureNames)==0:
        featureNames = map(str,indices)
    
    fN2 = [featureNames[a] for a in indices]
    print("Feature ranking:")

    for f in range(len(indices)):
        print("%d. feature %d=%s (%f)" % (f + 1, indices[f], featureNames[indices[f]],featureImportances[indices[f]]))

    # Plot the feature importances of the forest
    # define a cutoff in terms of feature_importance
    if nfeatures <= 30:
        kfeatures = nfeatures # keep all if smaller than 30
    else:
        kfeatures = 30
        
    kindices = indices[:kfeatures]
    plt.title("Feature importances")
    plt.barh(range(len(kindices)), featureImportances[kindices],
       color="steelblue", xerr=std[kindices], align="center",ecolor='k')#,lw=2)
    plt.yticks(range(len(kindices)),fN2)
    #grid(True)
    c1 = 'value'
    c2 = 'std'
    tdata = np.vstack([featureImportances[indices],std[indices]])
    df = pd.DataFrame(data = tdata.T,index=fN2,columns=[c1,c2])
    return df
                      #indices, std
    #xticks(range(len(indices)), indices)
#xlim([-1, 10])

### push all of the feature importances to a data frame

In [ ]:
fivehund_5= plotFI(kf_fits500[5],df.columns)

In [ ]:
fivehund_5.head()

In [ ]:
five_hundred_feature_importances = []
for mdl in kf_fits500:
    five_hundred_feature_importances.append(plotFI(mdl,df.columns))

In [ ]:
fhfeature = pd.concat(five_hundred_feature_importances,axis=1)

In [ ]:
fhfeature.columns = ['M1value','M1std','M2value','M2std','M3value','M3std','M4value','M4std','M5value','M5std',
                     'M6value','M6std','M7value','M7std','M8value','M8std','M9value','M9std','M10value','M10std']
fhfeature.head()

In [ ]:
value_columns = [x for x in fhfeature.columns if x.endswith('value')]
fhfeature[value_columns].mean(axis=1).order()

In [ ]:
fhfeature[value_columns].max()

In [ ]:
for mx in value_columns:
    print fhfeature[fhfeature[mx] == fhfeature[mx].max()].index

In [ ]:
df.HAVE_INS.value_counts()

In [ ]:
empl_active.HAVE_INS.value_counts(), empl_sep.HAVE_INS.value_counts()

In [ ]:
[kf_fits500[a].predict_proba(activX[1]) for a in xrange(0,len(kf_fits500))]

In [ ]:
[kf_fits500[a].predict_proba(separX[1]) for a in xrange(0,len(kf_fits500))]

#### does a great job of predicitng the classes

In [ ]:
kf_fits500[5].feature_importances_[indxa]

In [ ]:
def rf_feature_importances(forest,column_names):
    importances= forest.feature_importances_
    importances_std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
    indices = np.argsort(importances)[::-1]
    feature_names = column_names[indices].tolist()
    return 

In [ ]:
plotFI(kf_fits[0],df.columns)
    

In [ ]:
plotFI(kf_fits[1],df.columns)

In [ ]:
plotFI(kf_fits[2],df.columns)

In [ ]:
y_prob = np.zeros((len(y),2))
for ii, jj in kf:
        X_train, X_test = X[ii], X[jj]
        y_train = y[ii]
        clf = clf_class(**kwargs)
        clf.fit(X_train,y_train)
        y_prob[jj] = clf.predict_proba(X_test)
        # so the prediction for the jj_th row is the prediction against the other set.
    return y_prob

In [ ]:
y_prob = np.zeros((len(y),2))
for idx,(ii, jj) in enumerate(kf_rfc):
    #print len(ii), len(jj)#, idx
    X_train, X_test = X[ii],X[jj]
    #y_train = y[ii]
    print idx,"\t",kf_fits[idx].score(X_test,y[jj])
    y_prob[jj] = kf_fits[idx].predict_proba(X_test)

In [ ]:
def plot_roc_curve(target_test, target_predicted_proba):
    fpr, tpr, thresholds = metrics.roc_curve(target_test, target_predicted_proba[:, 1])
    
    roc_auc = metrics.auc(fpr, tpr)
    # Plot ROC curve
    plt.plot(fpr, tpr, label='ROC curve (area = %0.3f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')  # random predictions curve
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate or (1 - Specifity)')
    plt.ylabel('True Positive Rate or (Sensitivity)')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")

In [ ]:
10*(len(jj)+1)-1 == len(df)

In [ ]:
plot_roc_curve(y,y_prob)

In [ ]:
empl_active.shape, df.shape

In [ ]:
empl_sep  =empl3[empl3.status == 1].copy()
empl_sep.shape

In [ ]:
#separX = empl_sep[df.columns].as_matrix().astype(np.float) # DONT" DO THIS I scaled X first; use the following assignment
separX = X[empl_sep.index,:]
activX = X[empl_active.index,:]

In [ ]:
#activX = empl_active[df.columns].as_matrix().astype(np.float)
activy = y[empl_active.index]# = df.as_matrix().astype(np.float)

In [ ]:
kf_fits[1].predict_proba(activX)[:,0]

In [ ]:
kf_fits[1].predict_proba(separX)[:,0]

In [ ]:
kf_rfc500,kf_fits500 = kfold_cv2(X,y,classifiers_list[-2],n_jobs=30,n_estimators=500)
print len(kf_fits500)

In [ ]:
%%time
kf_rfc1k,kf_fits1k= kfold_cv2(X,y,classifiers_list[-2],n_jobs=50,n_estimators=1000)
print len(kf_fits1k)

In [ ]:
y_prob_1k = np.zeros((len(y),2))
for idx,(ii, jj) in enumerate(kf_rfc1k):
    #print len(ii), len(jj)#, idx
    X_train, X_test = X[ii],X[jj]
    #y_train = y[ii]
    print idx,"\t",kf_fits1k[idx].score(X_test,y[jj])
    y_prob_1k[jj] = kf_fits1k[idx].predict_proba(X_test)
print "__________________________"
plot_roc_curve(y,y_prob_1k)

## for 1000 trees, get feature importances

In [ ]:
feature_importances_list_1k= []
for mdl in kf_fits1k:
    feature_importances_list_1k.append(plotFI(mdl,df.columns))
fi_1k_df = pd.concat(feature_importances_list_1k,axis=1)
fi_1k_df.columns = fhfeature.columns

In [ ]:
df.MIN_RT_ANNUAL.head()

### Look at the most important feature: 'FUNC_CNT'
* this is the number of employees with the same give function
* index # is 52
* range shown below
* plot the unscaled versus scaled
* plot histogram of values in active vs inactive

In [ ]:
fc_index = list(fi_1k_df.index).index('FUNC_CNT')

In [ ]:
print fc_index, fi_1k_df.ix[fc_index].name
X[:,52].min(), X[:,52].max()

In [ ]:
fc_scaled_range = [X[:,fc_index].min(),X[:,fc_index].max()]
fc_unscaled_range = [df['FUNC_CNT'].min(),df['FUNC_CNT'].max()]
#:,fc_index].min(),X[:,fc_index].max()]
print fc_scaled_range, fc_unscaled_range

In [ ]:
len(X[:,fc_index])
len(df['FUNC_CNT'].values)

In [ ]:
plt.scatter(df['FUNC_CNT'].values,X[:,fc_index],alpha=0.3)

### what??

In [ ]:
plt.hist(activX[:,52],range=fc_scaled_range,normed=True,bins=30)
plt.hist(separX[:,52],range=fc_scaled_range,normed=True,alpha=0.5,bins=30)

## try applying all models to the test case and checking result.
* predict is a nX1 array; value of 0 means class 0, value of 1 means class 1.
* predict proba is a nX2 array; 1st column is prediction for class 0.
* average over this value


In [ ]:
class0_pred = [rf_model.predict(activX) for rf_model in kf_fits]
avg_class0_pred_active = np.mean(np.vstack(class0_pred).T, axis=1)

In [ ]:
class0_pred_sep = [rf_model.predict(separX) for rf_model in kf_fits]
avg_class0_pred_sep = np.mean(np.vstack(class0_pred_sep).T, axis=1)

In [ ]:
min_class0_pred_sep = np.min(np.vstack(class0_pred_sep).T, axis=1)
min_class0_pred_sep.sum()

In [ ]:
class0_pred_sep

In [ ]:
empl_sep['status'].head()

In [ ]:
np.min(avg_class0_pred_sep)

In [ ]:
min_val = np.min([np.min(avg_class0_pred_active),np.min(avg_class0_pred_sep)])
max_val = np.max([np.max(avg_class0_pred_active),np.max(avg_class0_pred_sep)])
plt.hist(avg_class0_pred_active,bins=30,range=[min_val,max_val],color='indianred')
plt.hist(avg_class0_pred_sep,bins=30,range=[min_val,max_val],color='steelblue',alpha= 0.3)

## AH THIS IS WHAT I EXPECTED -- clear separation between the active and retired (see above figure)

# May 14, 2015
## Repeat RF with unscaled data

In [ ]:
df.shape, empl.shape, empl2.shape, empl3.shape

In [ ]:
u = empl3.status.as_matrix().astype(np.int)
df = empl3.drop(['status','sep_status','retired'],axis=1).copy()
V = df.as_matrix().astype(np.float)

In [ ]:
%%time
rf1k_kfold,rf1k_kfoldfits= kfold_cv2(V,u,classifiers_list[-2],n_jobs=50,n_estimators=1000)
print len(rf1k_kfold)

### Plot ROC curve and get AUC

In [ ]:
def return_kfold_proba(X,y,kfold_mdl,kfold_fits):
    y_proba = np.zeros((len(y),2))
    for idx,(ii,jj) in enumerate(kfold_mdl):
        X_train, X_test = X[ii],X[jj]
        print idx, "\t", kfold_fits[idx].score(X_test,y[jj])
        y_proba[jj] = kfold_fits[idx].predict_proba(X_test)
    print "++++++++++++++++++++++++++++++"
    return y_proba

In [ ]:
u_prob_1k = return_kfold_proba(V,u, rf1k_kfold,rf1k_kfoldfits)

plot_roc_curve(u,u_prob_1k)

In [ ]:
fi_1knew_list = []
for mdl in rf1k_kfoldfits:
    fi_1knew_list.append(plotFI(mdl,df.columns))
    
# join these together
fi_1krf_df = pd.concat(fi_1knew_list,axis=1)
fi_1krf_df.columns = fhfeature.columns

In [ ]:
activV = V[empl_active.index,:]
separV = V[empl_sep.index,:]

In [ ]:
df.FUNC_CNT.describe()

In [ ]:
empl3[empl3.FUNC_CNT == 1].status.value_counts()

In [ ]:
#plt.hist(activV[:,52],range=fc_unscaled_range,normed=True,bins=30)
#plt.hist(separV[:,52],range=fc_unscaled_range,normed=True,alpha=0.5,bins=30)

### try omitting FUNC_CNT and EXTFUNC_CNT

In [ ]:
df3 = empl3.drop(['status','sep_status','retired','FUNC_CNT','EXTFUNC_CNT'],axis=1).copy()
Z = df3.as_matrix().astype(np.float)

In [ ]:
%%time
rf500new,rf500fits= kfold_cv2(Z,u,classifiers_list[-2],n_jobs=50,n_estimators=500)
#print len(rf1k_kfold)

In [ ]:
u_prob500n = return_kfold_proba(Z,u, rf500new, rf500fits)

plot_roc_curve(u,u_prob500n)

In [ ]:
fi_500new_list = []
for mdl in rf500fits:
    fi_500new_list.append(plotFI(mdl,df3.columns))
    
# join these together
rffi_500n_df = pd.concat(fi_500new_list,axis=1)
rffi_500n_df.columns = fhfeature.columns

In [ ]:
rffi_500n_df[value_columns].sort([value_columns[0]],ascending=False).head(20)

In [ ]:
rffi_500n_df[value_columns].mean(axis=1).order().tail(20)

### so apply time-shift.

In [ ]:
temporal_cols_indices = [list(rffi_500n_df.index).index(tc) for tc in temporal_cols]
[(tc, list(rffi_500n_df.index).index(tc)) for tc in temporal_cols]

In [ ]:
for idx,tci in enumerate(temporal_cols_indices):
    print temporal_cols[idx],Z[:,tci].min(),Z[:,tci].max()

In [ ]:
activZ = Z[empl_active.index,:]


In [ ]:
temporal_cols_indices[:6]

In [ ]:
## time shift rules
tdelta = 3.0
p3_actZ = activZ
p3_actZ[:,temporal_cols_indices[:6]] = activZ[:,temporal_cols_indices[:6]]+12.0*tdelta
p3_actZ[:,[1,97]] = p3_actZ[:,[1,97]]+tdelta

In [ ]:

u_act = u[empl_active.index]


In [ ]:
u_active_pred3_a = rf500fits[0].pr

In [ ]:
u_active_pred_a = rf500fits[0].predict(p1_actZ)

In [ ]:
sum(u_active_pred_a)

In [ ]:
[rf500fits[a].predict_proba(p3_actZ[400])[:,0] for a in xrange(0,len(rf500fits))]

In [ ]:
[rf500fits[a].predict_proba(activZ[400])[:,0] for a in xrange(0,len(rf500fits))]

In [ ]:
## to combine the CV folds use , min, max, avg, median?
def generate_probabilies(X,model_list,my_class=0,method='avg'):
    class_proba = [mdl.predict_proba(X)[:,my_class] for mdl in model_list]
    gp_class_proba= np.vstack(class_proba).T
    if method == 'avg':
        agg_proba = np.mean(gp_class_proba,axis=1)
    elif method == 'max':
        agg_proba = np.max(gp_class_proba,axis=1)
    elif method == 'min':
        agg_proba = np.min(gp_class_proba,axis=1)
    elif method == 'median':
        agg_proba = np.median(gp_class_proba,axis=1)
    else:
        print "The method {0} has not been implemented yet".format(method)
        return 0
    return agg_proba


In [ ]:
act_avg_1yr = generate_probabilies(p1_actZ,rf500fits)

In [ ]:
act_avg_3yr = generate_probabilies(p3_actZ,rf500fits)

In [ ]:
act_avg_0yr = generate_probabilies(activZ,rf500fits)

In [ ]:
plt.scatter(act_avg_0yr,act_avg_1yr,label='1 year',alpha=0.5)
plt.scatter(act_avg_0yr,act_avg_3yr,label='3 year',color='indianred',alpha=0.3)
plt.xlabel('Avg Probability TODAY')
plt.ylabel('Avg Probability $n$ year')
plt.legend()

# some questions
1. how many people leave in a given year -- what is the turnover rate?
2. can I post-date the temporal components to create a validation set for a model I've made?
3. can I create a view of the predicted probability of remaining as a function of time?
    - is this just a poor-mans survival analysis?
4. what is the effect of restricting the feature space to mostly temporal features?

In [ ]:
emplxtra_df = pd.read_csv('employee_dataframe8.tsv',sep='\t')
emplxtra_df.shape

In [ ]:
sum(emplxtra_df.Age_years - empl3.Age_years) #confirms that the difference in age is zero

In [ ]:
emplxtra_df.HIRE_DT.head()


In [ ]:
emplxtra_df['hire_year'] = emplxtra_df['HIRE_DT'].apply(lambda x: int(str(x)[:4]))
emplxtra_df.hire_year.hist(bins=70)
plt.xlabel('Year of Hire')
plt.ylabel('Counts')
emplxtra_df.hire_year.describe()

In [ ]:
### Construct a dataframe of # active employees given the end of the month
monthly_range_terminations = pd.date_range(emplxtra_df.TERMINATION_DT.min(),emplxtra_df.TERMINATION_DT.max(),freq='M')


In [ ]:
monthly_range_terminations

In [ ]:
sum(emplxtra_df['HIRE_DT'].apply(lambda x: pd.to_datetime(x)) < monthly_range_terminations[0])

In [ ]:
# ccreate listing where they are timestamps for comparisions
empl_dates = pd.DataFrame()
empl_dates[['hire_tstmp','term_tstmp']] = emplxtra_df[['HIRE_DT','TERMINATION_DT']].apply(lambda x: pd.to_datetime(x))

In [ ]:
len(empl_dates[(empl_dates.hire_tstmp < monthly_range_terminations[0]) & (empl_dates.term_tstmp > monthly_range_terminations[0])])

In [ ]:
monthly_range_terminations[0]-1

In [ ]:
nactive = []
nsep2 = []
nhired = []
prev_date = monthly_range_terminations[0]-1
#nsep.append(0.0)
for idx, my_date in enumerate(monthly_range_terminations):
    nactive.append( len(empl_dates[(empl_dates.hire_tstmp < my_date) &(empl_dates.term_tstmp > my_date)]))
    if idx == 0:
        nsep2.append(len(empl_dates[(empl_dates.term_tstmp <= my_date) & (empl_dates.hire_tstmp < my_date)]))
        nhired.append( len(empl_dates[(empl_dates.hire_tstmp <= my_date) &(empl_dates.hire_tstmp > prev_date)]))
    else:
        nsep2.append(len(empl_dates[(empl_dates.term_tstmp <= my_date) & (empl_dates.hire_tstmp < my_date) &( empl_dates.term_tstmp > monthly_range_terminations[idx-1])]))
        nhired.append( len(empl_dates[(empl_dates.hire_tstmp <= my_date) &(empl_dates.hire_tstmp > monthly_range_terminations[idx-1])]))
        
    #    nsep.append(sum(empl_dates.term_tstmp <= my_date) - nsep[idx-1]) 
            #ntermed = sum (empl_dates.term

In [ ]:
plt.plot(monthly_range_terminations,nsep)
plt.xlabel('Date')
plt.ylabel('Worforce Cumulative Separations')

In [ ]:
plt.plot(monthly_range_terminations,nsep2)
plt.plot(monthly_range_terminations,nhired)
plt.xlabel('Date')
plt.ylabel('Worforce Separations')

In [ ]:
plt.plot(monthly_range_terminations,nactive)
plt.xlabel('Date')
plt.ylabel('Worforce Counts')

In [ ]:
workforce= pd.DataFrame(data =nactive,index=monthly_range_terminations,columns=['Active'])
delta = workforce.diff()
workforce['netChange'] = delta
workforce['Separations'] = nsep2
workforce['Hires'] = nhired
workforce.head()

In [ ]:
workforce.Active.head(12)

In [ ]:
workforce[workforce.Separations > 1].head()

In [ ]:
annualwf = pd.DataFrame()
annualwf = workforce[['Separations','Hires']].resample('A',how='sum')#.plot()
annualwf['Active'] =workforce['Active'].resample('A',how='last')
annualwf.head()

In [ ]:
annualwf['NetChange'] = annualwf.Active.diff()

In [ ]:
## restrict to after 1987
annualwf.ix['1988-1-31':,[0,1,3]].plot()
plt.xlabel('Date')
plt.axhline(0,color='k')

In [ ]:
annualwf['TurnoverRate'] = annualwf['Separations']/annualwf['Active']*100.

## Working on post-dating my dataset
plan is to take a snapshot from year-end 2013 (so I know the outcome of status after 1 year) and apply my model to this to test predictive accuracy.
* Requires:
    1. elimination of employees who started (HIRE_DT) after 2013-12-31
    2. subtraction of temporal feaures by 1 year for those remaining.
* use the date information loaded for annualwf above.

In [ ]:
test_date1 = pd.to_datetime('2014-01-01')
set2014active_idx = empl_dates[(empl_dates.hire_tstmp < test_date1) &(empl_dates.term_tstmp > test_date1)].index

In [ ]:
emplxtra_df.ix[set2014active_idx].status.value_counts()

In [ ]:
zz_active_true = empl3.ix[set2014active_idx].status.as_matrix().astype(np.int)
#u = empl3.status.as_matrix().astype(np.int)
#y = empl3['Tenure_years'].values.astype(np.float)

In [ ]:
ZZ = Z[set2014active_idx,:]
zz_active_predproba0 = rf500fits[0].predict_proba(ZZ)

In [ ]:
plot_roc_curve(zz_active_true,zz_active_predproba0)

In [ ]:
zz_active_pred0 = rf500fits[0].predict(ZZ)
my_conf_matrix_2014 = metrics.confusion_matrix(zz_active_true,zz_active_pred0)
sns.heatmap(my_conf_matrix_2014, annot=True,  fmt='')
plt.title('Random Forest Classifier Confusion Matrix')

In [ ]:
metrics.matthews_corrcoef(zz_active_true, zz_active_pred0), metrics.accuracy_score(zz_active_true, zz_active_pred0)

In [ ]:
print (metrics.classification_report(zz_active_true, zz_active_pred0,target_names=['current','separated']))

In [ ]:
empl3.ix[set2014active_idx].status.value_counts()

In [ ]:
#dumb-classifier
zz_all_stay = np.zeros(len(zz_active_true))
print (metrics.classification_report(zz_active_true,zz_all_stay))
sns.heatmap(metrics.confusion_matrix(zz_active_true,zz_all_stay), annot=True,  fmt='')
plt.title('Random Forest Classifier Confusion Matrix: Dumb Model')

In [ ]:
plot_roc_curve(zz_active_true,np.vstack([zz_all_stay,np.ones(len(zz_all_stay))]).T)

In [ ]:
print [mdl.score(ZZ,zz_active_true) for mdl in rf500fits]
np.mean([mdl.score(ZZ,zz_active_true) for mdl in rf500fits])

## combining these estimators


In [ ]:
def combine_rf_estimators(rf_a,rf_b):
    rf_a.estimators_ += rf_b.estimators_
    rf_a.n_estimators = len(rf_a.estimators_)
    return rf_a

In [ ]:
rf500combo = reduce(combine_rf_estimators,rf500fits) # create a combined RF model

In [ ]:
rf500combo.score(ZZ,zz_active_true)

In [ ]:
zz_sep_indx = np.where(zz_active_true == 1)[0]

In [ ]:
plt.hist(rf500combo.predict_proba(ZZ)[zz_sep_indx,0])

In [ ]:
zz_active_pred_combo = rf500combo.predict(ZZ)
my_CM = metrics.confusion_matrix(zz_active_true,zz_active_pred_combo)
sns.heatmap(my_CM, annot=True,  fmt='')
plt.title('Random Forest Classifier Confusion Matrix')

## I legitimately fear overfitting
Let me try to reduce my dataset to employees prior to 2011, reset their status, train a model and then predict on employees from 2011 to present.

In [ ]:
test_date2 = pd.to_datetime('2010-01-01')
#test_date1 = pd.to_datetime('2014-01-01')
before2010_idx = empl_dates[(empl_dates.hire_tstmp < test_date2)].index# &(empl_dates.term_tstmp > test_date2)].index
after2010_idx = list(set((empl3.index))-set(before2010_idx))
print len(before2010_idx), len(after2010_idx)
print " this amounts to a hold-out fraction of {0}".format(len(after2010_idx)/float(len(empl3)))
#len(empl3.ix[~before2011_idx])

In [ ]:
# split into test-train split
train_df = empl3.ix[before2010_idx].copy()
test_df = empl3.ix[after2010_idx].copy()


In [ ]:
# reset the status of train_df if term_tstmp > test-date; also fix tenure/age, etc.
indices_to_fix = empl_dates[(empl_dates.hire_tstmp < test_date2) & (empl_dates.term_tstmp >= test_date2)].index
len(indices_to_fix)

In [ ]:
print empl3.status.value_counts(),
print empl3.ix[before2010_idx].status.value_counts()
print empl3.ix[indices_to_fix].status.value_counts()

In [ ]:
train_df.loc[indices_to_fix,'status'] = 0
train_df.status.value_counts()

In [ ]:
empl_dates.ix[46]

## try to clean up time-sensitive data in train_df
* leave MOS; just change Tenure & Age

In [ ]:
train_df[['Tenure_years','Age_years']].ix[indices_to_fix]

In [ ]:
y_train = train_df.status.as_matrix().astype(np.int)
df = train_df.drop(['status','sep_status','retired'],axis=1).copy()
V = df.as_matrix().astype(np.float)

In [ ]:
%%time 
rfc_500trees = ensemble.RandomForestClassifier(n_estimators=500,n_jobs=50)
CVscores = cross_validation.cross_val_score(rfc_500trees, V, y_train, cv=10)
print CVscores
CVscores.mean()

In [ ]:
myRFC = rfc_500trees.fit(V,y_train)

In [ ]:
y_test = test_df.status.as_matrix().astype(np.int)
V_test = test_df.drop(['status','sep_status','retired'],axis=1).as_matrix().astype(np.float)

In [ ]:
## apply this to the test-set
v_test_pred = rfc_500trees.predict(V_test)
v_test_predproba = rfc_500trees.predict_proba(V_test)

In [ ]:
## apply this to the test-set
v_test_pred1 = myRFC.predict(V_test)
v_test_predproba1 = myRFC.predict_proba(V_test)

In [ ]:
my_conf_matrix = metrics.confusion_matrix(y_test,v_test_pred1)
sns.heatmap(my_conf_matrix, annot=True,  fmt='')
plt.title('Random Forest Classifier Confusion Matrix: built on pre 2010 data')

In [ ]:
plot_roc_curve(y_test,v_test_predproba1)

In [ ]:
myRFC.score(V_test,y_test)

In [ ]:
plotFI(myRFC,df.columns)

### how to combine the estimators

In [ ]:
kf_fits_combo = reduce(combine_rf_estimators,kf_fits)

In [ ]:
kf_fits_combo.score(X,y)

In [ ]:
np.sum(kf_fits_combo.predict(separX))

In [ ]:
class0_proba = [rf_model.predict_proba(activX)[:,0] for rf_model in kf_fits]

In [ ]:
class0_proba_sep = [rf_model.predict_proba(separX)[:,0] for rf_model in kf_fits]
avg_sep_proba = np.mean(np.vstack(class0_proba_sep).T,axis=1)

In [ ]:
avg_proba = np.mean(np.vstack(class0_proba).T,axis=1)
avg_proba

In [ ]:
plt.plot(activy,avg_proba, marker='d',alpha=0.3)
plt.plot(y[empl_sep.index],avg_sep_proba,marker='h',alpha=0.2)

In [ ]:
[x.T for x in class0_proba]

In [ ]:
avg_pred

In [ ]:
#from sklearn.cross_validation import cross_val_score
rfc_500trees = ensemble.RandomForestClassifier(n_estimators=500,n_jobs=30)
CVscores = cross_validation.cross_val_score(rfc_500trees, X, y, cv=10)
CVscores

In [ ]:
CVscores.mean()

In [ ]:
[(clf,metrics.accuracy_score(y,kfold_cv(X,y,clf))) for clf in classifiers_list]

In [ ]:
### Gradient Boosted Trees

In [ ]:
ensemble.GradientBoostingClassifier?
#r(n_estimators=100, max_depth=3, loss='ls')

### define classes as [0,1,2,3,4,5,6-10,11-15,over]

In [ ]:
y_tenure_class = pd.cut(empl2.Tenure_years,[-1,0,1,2,3,4,5,10,15,200],labels=False)#,labels=[1,2,3,4,5,6,7,8])#.values
y_tenure_class

In [ ]:
df = empl2.drop(['Tenure_years','status','sep_status','retired'],axis=1).copy()
X = df.as_matrix().astype(np.float)
y_tenure_class.shape, X.shape

In [ ]:
tenure_classes = list(set(list(y_tenure_class)))
print tenure_classes
[(x,list(y_tenure_class).count(x)) for x in tenure_classes]
#list(y_tenure_class).count(1)

In [ ]:
from sklearn import cross_validation
from sklearn import tree
from sklearn import svm
from sklearn import ensemble
from sklearn import neighbors
from sklearn import linear_model
from sklearn import metrics

In [ ]:
def kfold_cv(X, y, clf_class, shuffle=True, n_folds=10, **kwargs):
    k_fold = cross_validation.KFold(len(y), n_folds=n_folds, shuffle=shuffle)
    y_pred = y.copy()
    for ii, jj in k_fold:
        X_train, X_test = X[ii], X[jj]
        y_train = y[ii]
        clf = clf_class(**kwargs)
        clf.fit(X_train,y_train)
        y_pred[jj] = clf.predict(X_test)
    return y_pred

In [ ]:
def kfold_cv_proba(X, y, clf_class, shuffle=True, n_folds=10, **kwargs):
    k_fold = cross_validation.KFold(len(y), n_folds=n_folds, shuffle=shuffle)
    y_prob = np.zeros((len(y),2))
    for ii, jj in k_fold:
        X_train, X_test = X[ii], X[jj]
        y_train = y[ii]
        clf = clf_class(**kwargs)
        clf.fit(X_train,y_train)
        y_prob[jj] = clf.predict_proba(X_test)
    return y_prob

In [ ]:
print('Dump Classifier: {:.2f}'.format(metrics.accuracy_score(y_tenure_class, [0 for ii in y_tenure_class.tolist()])))

In [ ]:
%%time
print('K Nearest Neighbor Classifier: {:.2f}'.format(metrics.accuracy_score(y_tenure_class, kfold_cv(X, y_tenure_class, neighbors.KNeighborsClassifier))))

In [ ]:
k_neighbors_conf_matrix = metrics.confusion_matrix(y_tenure_class, kfold_cv(X, y_tenure_class, neighbors.KNeighborsClassifier))

In [ ]:
sns.heatmap(k_neighbors_conf_matrix, annot=False)

In [ ]:
# truth: 
[(a,sum(y_tenure_class==a)) for a in tenure_classes]

In [ ]:
pred_tenure_class = kfold_cv(X,y_tenure_class, neighbors.KNeighborsClassifier)
len(pred_tenure_class)

In [ ]:
np.shape(k_neighbors_conf_matrix)

In [ ]:
k_neighbors_conf_matrix[:,0]

In [ ]:
k_neighbors_conf_matrix

## well this is OK, but not great

In [ ]:
print('K Nearest Neighbor Classifier:\n {}\n'.format(metrics.classification_report(y_tenure_class, kfold_cv(X, y_tenure_class, neighbors.KNeighborsClassifier))))

In [ ]:
#y_tenure = empl2.Tenure_years.apply(lambda x: np.round(x,0))
#y_tenure.value_counts()

### since the goal is to determine number of separation in a future time point, maybe I should create a target that is the number of tenure years.

## first test is to predict on separation at all (status)

In [ ]:
y = empl.status.as_matrix().astype(np.int)
df = empl.drop(['status','sep_status','retired'],axis=1).copy()
X = df.as_matrix().astype(np.float)

In [ ]:
y.shape, X.shape

So I have 199419 instances and 203 features

####Next scale them by removing mean and dividing by standar deviation.

In [ ]:
scaler = preprocessing.StandardScaler()
X = scaler.fit_transform(X)


In [ ]:
from sklearn import cross_validation

In [ ]:
cross_validation.KFold?

### use K-fold Cross-validation --> my classes are not too unbalanced

In [ ]:
def kfold_cv(X, y, clf_class, shuffle=True, n_folds=10, **kwargs):
    k_fold = cross_validation.KFold(len(y), n_folds=n_folds, shuffle=shuffle)
    y_pred = y.copy()
    for ii, jj in k_fold:
        X_train, X_test = X[ii], X[jj]
        y_train = y[ii]
        clf = clf_class(**kwargs)
        clf.fit(X_train,y_train)
        y_pred[jj] = clf.predict(X_test)
    return y_pred

In [ ]:
np.shape(y)

In [ ]:
kf = cross_validation.KFold(len(y),n_folds =10,shuffle=True)


In [ ]:
kfold_cv(X,y,linear_model.PassiveAggressiveClassifier)

In [ ]:
from sklearn import cross_validation
from sklearn import tree
from sklearn import svm
from sklearn import ensemble
from sklearn import neighbors
from sklearn import linear_model
from sklearn import metrics

In [ ]:
print('Passive Aggressive Classifier: {:.2f}'.format(metrics.accuracy_score(y, kfold_cv(X, y, linear_model.PassiveAggressiveClassifier))))
print('Gradient Boosting Classifier:  {:.2f}'.format(metrics.accuracy_score(y, kfold_cv(X, y, ensemble.GradientBoostingClassifier))))
print('Support vector machine(SVM):   {:.2f}'.format(metrics.accuracy_score(y, kfold_cv(X, y, svm.SVC))))
print('Random Forest Classifier:      {:.2f}'.format(metrics.accuracy_score(y, kfold_cv(X, y, ensemble.RandomForestClassifier))))
print('K Nearest Neighbor Classifier: {:.2f}'.format(metrics.accuracy_score(y, kfold_cv(X, y, neighbors.KNeighborsClassifier))))
print('Logistic Regression:           {:.2f}'.format(metrics.accuracy_score(y, kfold_cv(X, y, linear_model.LogisticRegression))))

In [ ]:
print('Dump Classifier: {:.2f}'.format(metrics.accuracy_score(y, [0 for ii in y.tolist()])))

## Confusion Matrix

In [ ]:
pass_agg_conf_matrix = metrics.confusion_matrix(y,kfold_cv(X, y, linear_model.PassiveAggressiveClassifier))
grad_ens_conf_matrix = metrics.confusion_matrix(y, kfold_cv(X, y, ensemble.GradientBoostingClassifier))
decision_conf_matrix = metrics.confusion_matrix(y, kfold_cv(X, y, tree.DecisionTreeClassifier))
ridge_clf_conf_matrix = metrics.confusion_matrix(y, kfold_cv(X, y, linear_model.RidgeClassifier))
svm_svc_conf_matrix = metrics.confusion_matrix(y, kfold_cv(X, y, svm.SVC))
random_forest_conf_matrix = metrics.confusion_matrix(y, kfold_cv(X, y, ensemble.RandomForestClassifier))
k_neighbors_conf_matrix = metrics.confusion_matrix(y, kfold_cv(X, y, neighbors.KNeighborsClassifier))
logistic_reg_conf_matrix = metrics.confusion_matrix(y, kfold_cv(X, y, linear_model.LogisticRegression))
dumb_conf_matrix = metrics.confusion_matrix(y, [0 for ii in y.tolist()]); # ignore the warning as they are all 0

conf_matrix = {
                1: {
                    'matrix': pass_agg_conf_matrix,
                    'title': 'Passive Aggressive',
                   },
                2: {
                    'matrix': grad_ens_conf_matrix,
                    'title': 'Gradient Boosting',
                   },
                3: {
                    'matrix': decision_conf_matrix,
                    'title': 'Decision Tree',
                   },
                4: {
                    'matrix': ridge_clf_conf_matrix,
                    'title': 'Ridge',
                   },
                5: {
                    'matrix': svm_svc_conf_matrix,
                    'title': 'Support Vector Machine',
                   },
                6: {
                    'matrix': random_forest_conf_matrix,
                    'title': 'Random Forest',
                   },
                7: {
                    'matrix': k_neighbors_conf_matrix,
                    'title': 'K Nearest Neighbors',
                   },
                8: {
                    'matrix': logistic_reg_conf_matrix,
                    'title': 'Logistic Regression',
                   },
                9: {
                    'matrix': dumb_conf_matrix,
                    'title': 'Dumb',
                   },
}

In [ ]:
fix, ax = plt.subplots(figsize=(16, 12))
plt.suptitle('Confusion Matrix of Various Classifiers')
for ii, values in conf_matrix.items():
    matrix = values['matrix']
    title = values['title']
    plt.subplot(3, 3, ii) # starts from 1
    plt.title(title);
    sns.heatmap(matrix, annot=True,  fmt='');

## get hire year

In [ ]:
atime = empl_bene['HIRE_DT'].ix[0]
str(atime)

In [ ]:
str(atime)[5:7]

## how many are there if we require hire_date > some date?


In [ ]:
empl_bene.HIRE_DT.ix[0]

In [ ]:
test_hire_date = pd.to_datetime('2002-11-30') #datetime(2002,11,30)
test_hire_date

In [ ]:
emplB = empl_bene[empl_bene.HIRE_DT > test_hire_date].copy()
print len(emplB)
print emplB.status.value_counts()
print emplB.sep_status.value_counts()


In [ ]:
#test_term_date = #test_hire_date
test_term_date = pd.to_datetime('2002-12-31')
emplC= empl[empl.TERMINATION_DT > test_term_date].copy()
print len(emplC)
print emplC.status.value_counts()
print emplC.sep_status.value_counts()

In [ ]:
emplC.RATE1.value_counts()

In [ ]:
def split_rate_column(x):
    list_of_defined_values = ['R1C1','R1C2','R1C3',
                              'R2C1','R2C2','R2C3',
                              'R3C1','R3C2','R3C3']
    #rcol=[]
    #ccol=[]
    if x in list_of_defined_values:
        rval = int(x[1])#col.append(x[1])
        cval = int(x[3]) #col.append(x[3])
        #print x[1],x[3]
    else:
        #rcol.append(0)
        #ccol.append(0)
        rval = 0
        cval = 0
    return rval,cval

In [ ]:
#rcol,ccol = 
input_column = 'RATE1'
rval = [x[0] for x in emplC[input_column].apply(lambda(x): split_rate_column(x)).as_matrix()]
cval = [x[1] for x in emplC[input_column].apply(lambda(x): split_rate_column(x)).as_matrix()]

In [ ]:
emplC[input_column+'_R'] = rval
emplC[input_column+'_C'] = cval
emplC[['RATE1','RATE1_R','RATE1_C']].head()

In [ ]:
def convert_rate_column(df,input_column):
    rval = [x[0] for x in df[input_column].apply(lambda(x): split_rate_column(x)).as_matrix()]
    cval = [x[1] for x in df[input_column].apply(lambda(x): split_rate_column(x)).as_matrix()]
    df[input_column+'_R'] = rval
    df[input_column+'_C'] = cval
    df.drop(input_column,inplace=True,axis=1)
    return df

In [ ]:
### convert the RATINGS columns
for col in ['RATE1','RATE2','RATE3','RATE4','RATE5','RATE6','RATE7','RATE8','RATE9','RATE10']:
    print col
    emplC = convert_rate_column(emplC,col)

In [ ]:
emplC.shape

## begin to Standardize some of these other columns



In [ ]:
empltbl3.EMPL_CLASS.value_counts(), empltbl3.EMPL_CLASS.unique()

In [ ]:
empltbl3.EMPL_TYPE.value_counts()

In [ ]:
pd.crosstab(empltbl3.EMPL_TYPE,empltbl3.EMPL_CLASS)

In [ ]:
pd.crosstab(empl.COMPANY,empl.EMPL_TYPE)

In [ ]:
# look at these distributions
plt.figure(figsize=(12,8))
emplC[emplC.status==0].Age_years.hist(bins=70,range=[15,84],color='darkred',label='current',alpha=0.8,normed=True)
emplC[emplC.status==1].Age_years.hist(bins=70,range=[15,84],color='dodgerblue',label='former',alpha=0.5,normed=True)
plt.legend()
plt.xlabel('Age in years')
plt.ylabel('Normed Counts')
plt.title('Distribution of Worforce Ages')

In [ ]:
# look at these distributions
plt.figure(figsize=(12,8))
emplB[emplB.status==0].Tenure_years.hist(bins=36,range=[0,35],color='darkred',label='current',alpha=0.8,normed=True)
emplB[emplB.status==1].Tenure_years.hist(bins=36,range=[0,35],color='dodgerblue',label='former',alpha=0.5,normed=True)
plt.legend()
plt.xlabel('Age in years')
plt.ylabel('Normed Counts')
plt.title('Distribution of Worforce Ages')

In [ ]:
empl_bene.zip5.value_counts()

In [ ]:
empl_bene.COMP_FREQUENCY.head()

In [ ]:
##
empl_bene['COMP_FREQ_ANNUAL'] = 1
empl_bene.loc[empl_bene['COMP_FREQUENCY']=='H','COMP_FREQ_ANNUAL'] = 0
empl_bene.drop('COMP_FREQUENCY',axis=1,inplace=True)

## Save this dataframe as a tab separated file

In [ ]:
outfile1 = 'employee_dataframe.tsv'
save_dataframe_file = True
if save_dataframe_file:
    empltbl3.to_csv(outfile1,sep='\t')

## convert datetime64[ns] format to a string ---> YYYY-MM-DD 
use date_columns

In [ ]:
for c in date_columns:
    empltbl3[c]= empltbl3[c].apply(lambda x: str(x).split(' ')[0])

In [ ]:
for col in empltbl3.columns:
    my_dtype = empltbl3[col].dtype
    if my_dtype != 'object' and my_dtype != 'float64' and my_dtype != 'int64':
        print col, "\t", empltbl3[col].dtype, "\t",empltbl3[col].ix[8] 

In [ ]:
### try converting the timedeltas to floats
empltbl3['Age_tdelta'].ix[3].astype('float')/(60*60*24*10**9)

In [ ]:
nano2days = 60.*60.*24.*10**9
empltbl3['Age_days'] = empltbl3['Age_tdelta'].apply(lambda x: x.astype('float')/(nano2days))
empltbl3['Tenure_days'] = empltbl3['Tenure_tdelta'].apply(lambda x: x.astype('float')/(nano2days))

In [ ]:
empltbl3.drop(['Age_tdelta','Tenure_tdelta'], axis=1,inplace=True)
empltbl3.shape

In [ ]:
len(empltbl3.Age_days.unique()), len(empltbl3.Tenure_days.unique())

In [ ]:
empltbl.Age_years.ix[:10].apply(lambda x: int(np.round(x,0)))

In [ ]:
# Keep just the year
empltbl_dates = empltbl[date_columns].copy() # save the dates.
for c in date_columns:
    #print c
    if c.endswith('DT'):
        c2 = c[:-2]+'YEAR'
    else:
        c2 = c[:-4]+'_YEAR'
    #print c2
    empltbl[c2]= empltbl[c].apply(lambda x: x[:4])

In [ ]:
empltbl.drop(date_columns,axis=1,inplace=True)

In [ ]:
empltbl['Age_year_int'] = empltbl['Age_years'].apply(lambda x: int(np.round(x,0)))
empltbl['Tenure_year_int'] = empltbl['Tenure_months'].apply(lambda x: int(np.round(x/12,0)))

In [ ]:
len(empltbl.Age_year_int.unique()), len(empltbl.Tenure_year_int.unique())

In [ ]:
empltbl.Tenure_year_int.value_counts().plot(kind='bar',color='darkslateblue')

In [ ]:
#now save this version for H2O
outfile2 = 'employee_dataframe2.ssv'
save_dataframe_file = True
if save_dataframe_file:
    empltbl.to_csv(outfile2,sep=';')

In [ ]:
empltbl.shape

In [ ]:
# duplicated employee keys:
#[empltbl.KEY.value_counts() > 1]
dup_empl_keys = []
empltblKEYcounts = empltbl.KEY.value_counts()
for a in empltblKEYcounts[empltblKEYcounts>1].index:
    print a
    dup_empl_keys.append(a)

for empkey in dup_empl_keys:
    print empltbl[empltbl.KEY == empkey]

## Continue to Refine the input data

In [ ]:
sum(empltbl3.zip5.isnull())

In [ ]:
empltbl3[empltbl3.zip5=='SW18 '].values

In [ ]:
empltbl3[empltbl3['STATE'].isnull()][['POSTAL_SFI','zip5','LOC_STATE','ADDRESS1']]

In [ ]:
sdf = summarize_dataframe2(empltbl3)

## Examine cleaning up the different feature (covariate) columns for anylysis on them.


### restrict the dataframe to do linear-regression on.
* first attempt is _ad hoc_ reduction of 185 columns
    * keep age_years, tenure_months, status, sex for sure
* look at cases where there is 'significant' correlation between columns:
    * for example JOBCNTx, LOCCNTx, DEPTCNTx, ...

In [ ]:
# define the set of columns that have missing values smaller than some threshold 
sdf['x_missing'] = sdf['nmissing']/len(empltbl3)
len(sdf[sdf['x_missing'] > 0]), len(sdf)

In [ ]:
sdf['y_arity'] = sdf['arity']/len(empltbl)
sdf['y_arity'].hist(bins=30,color='deepskyblue')

In [ ]:
sdf['x_missing'].hist(bins=30,color='deeppink')

In [ ]:
sdf.to_csv('employee_table_summary_3.csv')
# save to file

In [ ]:
len(sdf)

In [ ]:
# now apply some logic to these columns for keeping and not
columns_to_fix_missing = []
columns_to_fix_NY = []
columns_to_omit = []

In [ ]:
mgrA_sf_df['managers']

In [ ]:
fig = plt.figure(figsize=(12.5,5))

kmf_mgr1=KaplanMeierFitter()

kmf_mgr1.fit(vc_empl4[mgrA].Tenure_months, timeline=tenure_month_range, event_observed=vc_empl4[mgrA].status,label='managers')
mgrA_sf_df = pd.DataFrame(kmf_mgr1.survival_function_)
ax = fig.add_subplot(111)
kmf_mgr1.plot(ax=ax,c='darkgreen')
kmf_mgr1.fit(vc_empl4[~mgrA].Tenure_months, timeline=tenure_month_range,event_observed=vc_empl4[~mgrA].status,label='non-managers')
kmf_mgr1.plot(ax=ax,color='darkslategray')
mgrA_sf_df = pd.concat([mgrA_sf_df,kmf_mgr1.survival_function_],axis=1)
plt.ylim(0,1.05)
plt.ylabel('Survival Distribution Function')
plt.xlabel('Tenure Months')

In [ ]:
mgrA_surv_prob = calc_future_survival_proba(vc_empl4,mgrA,mgrA_sf_df['managers'],future_years_list)

In [ ]:
plt.plot(future_years_list,mgrA_surv_prob.sum()/len(mgrA_surv_prob),'*:',color='steelblue')

In [ ]:
mgrA_surv_prob.sum()/len(mgrA_surv_prob)

In [ ]:
ues = vc_empl4.GRADE=='UES'
print sum(ues)

In [ ]:
pd.crosstab(vc_empl4.GRADE,vc_empl4.status)

In [ ]:
fig = plt.figure(figsize=(12.5,5))

kmf_ues=KaplanMeierFitter()

kmf_ues.fit(vc_empl4[ues].Tenure_months, timeline=tenure_month_range, event_observed=vc_empl4[ues].status,label='UES')
tmp_df = pd.DataFrame(kmf_ues.survival_function_)
ax = fig.add_subplot(111)
kmf_ues.plot(ax=ax,c='darkorange')
kmf_ues.fit(vc_empl4[~ues].Tenure_months, timeline=tenure_month_range,event_observed=vc_empl4[~ues].status,label='non-UES')
kmf_ues.plot(ax=ax,color='darkslategray')
tmp_df = pd.concat([tmp_df,kmf_ues.survival_function_],axis=1)
plt.ylim(0,1.05)
plt.ylabel('Survival Distribution Function')
plt.xlabel('Tenure Months')

In [ ]:
tmp_df.head()

In [ ]:
vc_empl4[ues][vc_empl4['status']==0].head()

In [ ]:
len(tmp_df)

In [ ]:
#kmf_grade.survival_function_
#tmp_df = pd.DataFrame(kmf_grade.survival_function_)
ues_surv_prob = calc_future_survival_proba(vc_empl4,ues,tmp_df['UES'],future_years_list)
ues_surv_prob.sum()/len(vc_empl4[ues][vc_empl4.status==0])

In [ ]:
ues_surv_prob.sum()/sum(ues)

In [ ]:
empltbl4.columns

In [ ]:
vc_empl4.columns

In [ ]:
# encode the VOLINVOL column as ['INVOLUNTARY':2,'VOLUNTARY':1,'NOT':0,'
empltbl4['sep_status'] = empltbl4['VOLINVOL'].replace({'NOT':0,'VOLUNTARY':1,'INVOLUNTARY':2,'OTHER':3,'UNKNOWN':3})
empltbl4.sep_status.value_counts()

In [ ]:
pd.crosstab(empltbl4.status,empltbl4.sep_status)

## Explore differences between voluntary/involuntary and Not 

In [ ]:
volunt = empltbl3[empltbl3.VOLINVOL=='VOLUNTARY'].copy()
current = empltbl3[empltbl3.VOLINVOL=='NOT'].copy()
involu = empltbl3[empltbl3.VOLINVOL=='INVOLUNTARY'].copy()
empl_df_list = [current,volunt,involu]
print [len(a) for a in empl_df_list]

In [ ]:
len(empltbl3[(empltbl3.VOLINVOL=='VOLUNTARY') |(empltbl3.VOLINVOL=='NOT')])

In [ ]:
empl_df_list_colors =['indianred','steelblue','darkkhaki']
empl_df_list_alphas =[1.0,0.7,0.4]
empl_df_list_label = ['current','voluntary','involuntary']

In [ ]:
## define histogram bin ranges
tenure_year_bin_range = [0,59]
age_year_bin_range = [15,84]

In [ ]:
plt.figure(figsize=(12,8))
for idx in xrange(0,len(empl_df_list)):
    my_color = empl_df_list_colors[idx]
    my_alpha = empl_df_list_alphas[idx]
    empl_df_list[idx].Age_years.hist(bins=70,range=age_year_bin_range,color=my_color,alpha=my_alpha,normed=True,label=empl_df_list_label[idx])
    plt.legend()
    plt.xlabel('Age in Years')
    plt.ylabel('Normed Counts')
    plt.title('Distribution of Workforce Ages')

In [ ]:
len(empltbl3[(empltbl3.VOLINVOL=='VOLUNTARY') & (empltbl3.Age_years > 54)])

In [ ]:
plt.figure(figsize=(12,8))
for idx in xrange(0,len(empl_df_list)):
    my_color = empl_df_list_colors[idx]
    my_alpha = empl_df_list_alphas[idx]
    empl_df_list[idx].Tenure_years.hist(bins=60,range=tenure_year_bin_range,color=my_color,alpha=my_alpha,normed=True,label=empl_df_list_label[idx])
    plt.legend()
    plt.xlabel('Tenure in Years')
    plt.ylabel('Normed Counts')
    plt.title('Distribution of Workforce Tenures')

In [ ]:
empltbl3.DIVISION_CODE_SFI.value_counts()

In [ ]:
pd.crosstab(empltbl3.DIVISION_CODE_SFI,empltbl3.COMPANY)#.value_counts()

In [ ]:
empltbl3[empltbl2.INTERN=='Y'].Tenure_months.hist(color='darkslategray',bins=80,normed=True)
empltbl3[empltbl2.INTERN=='N'].Tenure_months.hist(color='darkturquoise',bins=80,alpha=0.5, normed=True)

## Look at how many have missing SKEY
* only one in the current table.

In [ ]:
#len(current[current.SKEY.isnull()][['Age_years','Tenure_months','COMPANY','GRADE','ANNUAL_RT','KEY']])
ceo_key = current[current.SKEY.isnull()].KEY.values[0]
print ceo_key
print current[current.SKEY.isnull()][['Age_years','Tenure_months','COMPANY','GRADE','ANNUAL_RT','KEY']]

In [ ]:
len(empltbl3.KEY.unique()), len(empltbl3.SKEY.unique()),len(current.SKEY.unique())

In [ ]:
## How many are missing SKEY in retired.
sum(empltbl3[empltbl3.status==1].SKEY.isnull())

# do survival analysis for each GRADE
* column == 'GRADE' 
* arity = 164
* Define a min-threshold of GRADE to look at.


In [ ]:
empltbl3.GRADE.value_counts().hist(color='forestgreen',bins=30)

In [ ]:
## start with top 20 grades:
empltbl3.GRADE.value_counts()[:20]

## Survival analysis

In [ ]:
#use lifelines
from lifelines import KaplanMeierFitter
kmf0 = KaplanMeierFitter()

In [ ]:

plt.figure(figsize=(14,6))

kmf0.fit(empltbl3.Tenure_months, event_observed=empltbl3.status)
kmf0.plot(color='darkslategray')
plt.ylabel('Survival Distribution Function')
plt.xlabel('Tenure months')
plt.title('KaplanMeier Survival: ALL employees')

In [ ]:
## define the range so all are on the same scale
min_tenure_months= empltbl3.Tenure_months.min()
max_tenure_months = empltbl3.Tenure_months.max()
print min_tenure_months, max_tenure_months
tenure_month_range = np.linspace(min_tenure_months,max_tenure_months,66)
len(tenure_month_range)

In [ ]:
empltbl3.GRADE.value_counts()[:40]

In [ ]:
## replace nan with 'XXX'
empltbl3.GRADE.fillna('XXX',inplace=True)

In [ ]:
mg_codes = [x for x in list(empltbl3.GRADE.unique()) if  str(x).startswith('MG')]

In [ ]:
mg_codes = [x for x in list(empltbl3.GRADE.unique()) if  str(x).startswith('MG')]
clvl_codes = [x for x in list(empltbl3.GRADE.unique()) if str(x).startswith('C')]
ra_codes = [x for x in list(empltbl3.GRADE.unique()) if str(x).startswith('RA')]
re_codes = [x for x in list(empltbl3.GRADE.unique()) if str(x).startswith('RE')]
rd_codes = [x for x in list(empltbl3.GRADE.unique()) if str(x).startswith('RD')]
pa_codes = [x for x in list(empltbl3.GRADE.unique()) if str(x).startswith('PA')]
pb_codes = [x for x in list(empltbl3.GRADE.unique()) if str(x).startswith('PB')]
ma_codes = [x for x in list(empltbl3.GRADE.unique()) if str(x).startswith('MA')]
sf_codes = [x for x in list(empltbl3.GRADE.unique()) if str(x).startswith('SF')]
rb_codes = [x for x in list(empltbl3.GRADE.unique()) if str(x).startswith('RB')]
rc_codes = [x for x in list(empltbl3.GRADE.unique()) if str(x).startswith('RC')]
pc_codes = [x for x in list(empltbl3.GRADE.unique()) if str(x).startswith('PC')]
l_codes = [x for x in list(empltbl3.GRADE.unique()) if str(x).startswith('L')]
fa_codes = [x for x in list(empltbl3.GRADE.unique()) if str(x).startswith('FA')]
u_codes = [x for x in list(empltbl3.GRADE.unique()) if str(x).startswith('U')]
i_codes = [x for x in list(empltbl3.GRADE.unique()) if str(x).startswith('I')]
a_codes = [x for x in list(empltbl3.GRADE.unique()) if str(x).startswith('A')]
b_codes = [x for x in list(empltbl3.GRADE.unique()) if str(x).startswith('B')]
#ri_codes = [x for x in list(empltbl3.GRADE.unique()) if str(x).startswith('RI')]
# assign missing and non to non_codes
non_codes = ['NON','XXX']
m_codes = ['M1N','M2N','M3N','M10']
rx_codes = ['RF1', 'RF2', 'RF3', 'RG1', 'RG2', 'RG3', 'RG4', 'RH1', 'RH2', 'RH3', 'RH4', 
            'RI2', 'RI3', 'RJ2', 'RJ3', 'RJ4', 'RK3','RL1', 'RL2', 'RL3', 'RL4', 'RM2']
# combine
two_codes = [x for x in list(empltbl3.GRADE.unique()) if str(x).startswith('2')]
t_codes = [x for x in list(empltbl3.GRADE.unique()) if str(x).startswith('T')]
oth_codes = i_codes+two_codes+t_codes+a_codes
oth_codes.append('ZFL')


In [ ]:
grade_groupings_list_A = [mg_codes,clvl_codes,ma_codes,pa_codes,pb_codes,sf_codes, ra_codes,rd_codes,re_codes, rb_codes, rc_codes,
                          pc_codes,l_codes,fa_codes, u_codes,b_codes,non_codes,m_codes,rx_codes,oth_codes]
len(grade_groupings_list_A)


In [ ]:
e3_grade_function_crosstab = pd.crosstab(empltbl3.GRADE,empltbl3.JOB_FUNCTION)
e3_grade_function_crosstab.MGT 

In [ ]:
e3_grade_function_crosstab[e3_grade_function_crosstab.MGT>0] 

In [ ]:
e3_grade_function_crosstab[e3_grade_function_crosstab.PTB>0] 

In [ ]:
for g in grade_groupings_list_A:
    print sum(empltbl3.GRADE.isin(g)), g

In [ ]:
from itertools import chain
used_grades = list(chain.from_iterable(grade_groupings_list_A))
sum(empltbl3.GRADE.isin(used_grades)), len(empltbl3)

In [ ]:
sorted([x for x in empltbl3.GRADE.unique() if x.startswith('R')])

In [ ]:
empltbl3[~empltbl3.GRADE.isin(used_grades)].GRADE.value_counts()

In [ ]:
empltbl3.GRADE.value_counts()[40:85]

## now look at each of the top 10 grades:

In [ ]:
# create a dataFrame of these survival functions.


fig = plt.figure(figsize=(12.5,5))
#figsize(16,8)

top10grades = list(empltbl3.GRADE.value_counts()[:85].index)
kmf_grade=KaplanMeierFitter()

kmf_grade.fit(empltbl3.Tenure_months, timeline=tenure_month_range, event_observed=empltbl3.status,label='ALL')
grade_sf_df = pd.DataFrame(kmf_grade.survival_function_)
ax=fig.add_subplot(111)
kmf_grade.plot(ax=ax,c='darkslategray')
for grade in top10grades:
    
    #plt.figure(figsize=(20,10))
    em_grade = empltbl3[empltbl3.GRADE == grade]
    kmf_grade.fit(em_grade.Tenure_months, timeline=tenure_month_range,event_observed=em_grade.status,label=grade)
    if grade.startswith('MG'):
        kmf_grade.plot(ax=ax)
    grade_sf_df = pd.concat([grade_sf_df,kmf_grade.survival_function_],axis=1)
#kmf_gender.fit(female.Tenure_months,  timeline=tenure_month_range,event_observed=female.status)
#kmf_gender.plot(ax=ax, c='deeppink',label='Female')

plt.ylim(0,1.05)
plt.ylabel('Survival Distribution Function')
plt.xlabel('Tenure Months')


In [ ]:
grade_sf_df.columns

In [ ]:
mgr = ((empltbl3.GRADE == 'MG2') |(empltbl3.GRADE == 'MG3') | (empltbl3.GRADE == 'MG4'))
sum(mgr)

In [ ]:
mgr = empltbl3.GRADE.isin(mg_codes)
sum(mgr)

In [ ]:
from lifelines.statistics import logrank_test
summary, p_value, test_results = logrank_test(empltbl3[mgr].Tenure_months, empltbl3[~mgr].Tenure_months, empltbl3[mgr].status, empltbl3[~mgr].status, alpha=.99 )
print summary

In [ ]:
fig = plt.figure(figsize=(12.5,5))

kmf_grade=KaplanMeierFitter()

kmf_grade.fit(empltbl3[mgr].Tenure_months, timeline=tenure_month_range, event_observed=empltbl3[mgr].status,label='managers')
mgr_sf_df = pd.DataFrame(kmf_grade.survival_function_)
ax = fig.add_subplot(111)
kmf_grade.plot(ax=ax,c='darkred')
kmf_grade.fit(empltbl3[~mgr].Tenure_months, timeline=tenure_month_range,event_observed=empltbl3[~mgr].status,label='non-managers')
kmf_grade.plot(ax=ax,color='darkslategray')
mgr_sf_df = pd.concat([mgr_sf_df,kmf_grade.survival_function_],axis=1)
plt.ylim(0,1.05)
plt.ylabel('Survival Distribution Function')
plt.xlabel('Tenure Months')

## BIGGER QUESTION IS whether this tenure is descriptive or proscriptive


In [ ]:
empltbl3[mgr].Tenure_months.hist(normed=True,alpha=0.3)
empltbl3[mgr][empltbl3.status==0].Tenure_months.hist(normed=True,color='indianred',alpha=.3)

## Take the set of employees that don't leave and calculate out their survival function in the future
* look at [1,10] years in the future for each.
* sum over all
* also get lower CL and upper CL
* divide sum by count to get % remaining.


In [ ]:
mgr_sf_df.head(), len(mgr_sf_df)

In [ ]:
mgr_df = pd.DataFrame(empltbl3[mgr].Tenure_months)

In [ ]:
[x for x in empltbl3[mgr][empltbl3.status==0].Tenure_months+12.0]

In [ ]:
mgr_sf_df.ix[:339.36].index[-1], mgr_sf_df.ix[339.36:].index[0]

In [ ]:
locate_closest_indices(mgr_sf_df, 339.36)

In [ ]:
def locate_closest_indices(df,index_to_find):
    my_min = df.ix[:index_to_find].index[-1]
    my_max = df.ix[index_to_find:].index[0]
    #.index <= index_to_find
    #my_max = df.index >=index_to_find
    print index_to_find, my_min, my_max
    return my_min,my_max

In [ ]:
def lin_interpolate(df,index_to_find,ycol):
    #x1,x2 = locate_closest_indices(df,x)
    my_min = df.ix[:index_to_find].index[-1]
    my_max = df.ix[index_to_find:].index[0]
    deltax = my_max-my_min
    #print my_min, my_max
    y_1 = df[ycol].ix[my_min]
    y_2 = df[ycol].ix[my_max]
    #print y_1, y_2
    
    deltay=df[ycol].ix[my_max]- y_1
    slope = deltay/deltax
    #print slope, deltax, deltay
    my_value = y_1+slope*(index_to_find-my_min)
    return my_value

In [ ]:
mgr_sf_df.ix[mgr_sf_df.ix[:339.36].index[-1]:mgr_sf_df.ix[339.36:].index[0]]#.plot()#.interpolate()

In [ ]:
lin_interpolate(mgr_sf_df,339.36,'managers')

In [ ]:
index_to_find = 371.1

my_min = df['A'] <= value_to_find
Max = df['A'] >= value_to_find
idx_Min = df.ix[Min, 'A'].idxmax()
idx_Max = df.ix[Max, 'A'].idxmin()
df.ix[idx_Min:idx_Max, ['A','B']]

In [ ]:
# add 1 to 10 years in to future
future_years_list = np.arange(1,11)
future_years_list*12.0

In [ ]:
plus_1yr = [lin_interpolate(mgr_sf_df,x,'managers') for x in empltbl3[mgr][empltbl3.status==0].Tenure_months+12.0]

In [ ]:
len(plus_1yr)

In [ ]:
current_mgr.Tenure_months.values

In [ ]:
future_year

In [ ]:
## define a new data frame to hold the information
current_mgr = empltbl3[mgr][empltbl3.status==0].Tenure_months.copy()
#current_mgr.columns='Current_tenure'#,inplace=True)
new_df = pd.DataFrame()
for future_year in future_years_list:
    cname = 'plus_'+str(future_year)+'yr'
    survival_prob = [lin_interpolate(mgr_sf_df,x,'managers') for x in current_mgr.values+future_year*12.0]
    a_df = pd.DataFrame(survival_prob,columns=[cname])
    new_df = pd.concat([new_df,a_df],axis=1)
    
print new_df.shape

In [ ]:
new_df.head()

In [ ]:
new_df.sum()/len(new_df)

In [ ]:
hold_df = mgr_sf_df.copy()

In [ ]:
clvl_codes

In [ ]:
clvl = empltbl3.GRADE.isin(clvl_codes)
sum(clvl)

In [ ]:
sum(empltbl3[clvl].status==0)

In [ ]:
fig = plt.figure(figsize=(12.5,5))

kmf_grade=KaplanMeierFitter()

kmf_grade.fit(empltbl3[clvl].Tenure_months, timeline=tenure_month_range, event_observed=empltbl3[clvl].status,label='C-Level')
#grade_sf_df = pd.DataFrame(kmf_grade.survival_function_)
ax = fig.add_subplot(111)
kmf_grade.plot(ax=ax,c='darkred')
kmf_grade.fit(empltbl3[~clvl].Tenure_months, timeline=tenure_month_range,event_observed=empltbl3[~clvl].status,label='non C-level')
kmf_grade.plot(ax=ax,color='darkslategray')
plt.ylim(0,1.05)
plt.ylabel('Survival Distribution Function')
plt.xlabel('Tenure Months')

### look at different PA/RA

In [ ]:
ra = empltbl3.GRADE.isin(ra_codes)
print sum(ra), ra_codes

In [ ]:
rd = empltbl3.GRADE.isin(rd_codes)
print sum(rd), rd_codes
for code in rd_codes:
    print code,sum(empltbl3.GRADE == code)

In [ ]:
fig = plt.figure(figsize=(12.5,5))

kmf_grade=KaplanMeierFitter()

kmf_grade.fit(empltbl3[ra].Tenure_months, timeline=tenure_month_range, event_observed=empltbl3[ra].status,label='RA')
#grade_sf_df = pd.DataFrame(kmf_grade.survival_function_)
ax = fig.add_subplot(111)
kmf_grade.plot(ax=ax,c='darkred')
kmf_grade.fit(empltbl3[~ra].Tenure_months, timeline=tenure_month_range,event_observed=empltbl3[~ra].status,label='non RA')
kmf_grade.plot(ax=ax,color='darkslategray')
plt.ylim(0,1.05)
plt.ylabel('Survival Distribution Function')
plt.xlabel('Tenure Months')

In [ ]:
fig = plt.figure(figsize=(12.5,5))

kmf_grade=KaplanMeierFitter()

kmf_grade.fit(empltbl3[rd].Tenure_months, timeline=tenure_month_range, event_observed=empltbl3[rd].status,label='RD')
#grade_sf_df = pd.DataFrame(kmf_grade.survival_function_)
ax = fig.add_subplot(111)
kmf_grade.plot(ax=ax,c='darkred')
kmf_grade.fit(empltbl3[~rd].Tenure_months, timeline=tenure_month_range,event_observed=empltbl3[~rd].status,label='non RD')
kmf_grade.plot(ax=ax,color='darkslategray')
plt.ylim(0,1.05)
plt.ylabel('Survival Distribution Function')
plt.xlabel('Tenure Months')

In [ ]:
re = empltbl3.GRADE.isin(re_codes)
print sum(re), re_codes

In [ ]:
fig = plt.figure(figsize=(12.5,5))

kmf_grade=KaplanMeierFitter()

kmf_grade.fit(empltbl3[re].Tenure_months, timeline=tenure_month_range, event_observed=empltbl3[re].status,label='RE')
#grade_sf_df = pd.DataFrame(kmf_grade.survival_function_)
ax = fig.add_subplot(111)
kmf_grade.plot(ax=ax,c='darkred')
kmf_grade.fit(empltbl3[~re].Tenure_months, timeline=tenure_month_range,event_observed=empltbl3[~re].status,label='non RE')
kmf_grade.plot(ax=ax,color='darkslategray')
plt.ylim(0,1.05)
plt.ylabel('Survival Distribution Function')
plt.xlabel('Tenure Months')

In [ ]:
pa = empltbl3.GRADE.isin(pa_codes)
print sum(pa), pa_codes

In [ ]:
ma_codes

In [ ]:
empltbl3[empltbl3.GRADE=='MG2'].JOBCODE.value_counts()

In [ ]:
empltbl3.JOBCODE.value_counts()

In [ ]:
empltbl3[empltbl3['status'==1]].

In [ ]:
empltbl3.JOB_FUNCTION.value_counts()

In [ ]:
empl[empl.GRADE_MA1==1].status.value_counts()

In [ ]:
fig = plt.figure(figsize=(12.5,5))

kmf_grade=KaplanMeierFitter()

kmf_grade.fit(empltbl3[pa].Tenure_months, timeline=tenure_month_range, event_observed=empltbl3[pa].status,label='PA')
#grade_sf_df = pd.DataFrame(kmf_grade.survival_function_)
ax = fig.add_subplot(111)
kmf_grade.plot(ax=ax,c='darkred')
kmf_grade.fit(empltbl3[~pa].Tenure_months, timeline=tenure_month_range,event_observed=empltbl3[~pa].status,label='non PA')
kmf_grade.plot(ax=ax,color='darkslategray')
plt.ylim(0,1.05)
plt.ylabel('Survival Distribution Function')
plt.xlabel('Tenure Months')

In [ ]:
pb = empltbl3.GRADE.isin(pb_codes)
print sum(pb), pb_codes

In [ ]:
fig = plt.figure(figsize=(12.5,5))

kmf_grade=KaplanMeierFitter()

kmf_grade.fit(empltbl3[pb].Tenure_months, timeline=tenure_month_range, event_observed=empltbl3[pb].status,label='PB')
#grade_sf_df = pd.DataFrame(kmf_grade.survival_function_)
ax = fig.add_subplot(111)
kmf_grade.plot(ax=ax,c='darkred')
kmf_grade.fit(empltbl3[~pb].Tenure_months, timeline=tenure_month_range,event_observed=empltbl3[~pb].status,label='non PB')
kmf_grade.plot(ax=ax,color='darkslategray')
plt.ylim(0,1.05)
plt.ylabel('Survival Distribution Function')
plt.xlabel('Tenure Months')

In [ ]:
sf = empltbl3.GRADE.isin(sf_codes)
print sum(sf), sf_codes

In [ ]:
fig = plt.figure(figsize=(12.5,5))

kmf_grade=KaplanMeierFitter()

kmf_grade.fit(empltbl3[sf].Tenure_months, timeline=tenure_month_range, event_observed=empltbl3[sf].status,label='SF')
#grade_sf_df = pd.DataFrame(kmf_grade.survival_function_)
ax = fig.add_subplot(111)
kmf_grade.plot(ax=ax,c='darkred')
kmf_grade.fit(empltbl3[~sf].Tenure_months, timeline=tenure_month_range,event_observed=empltbl3[~sf].status,label='non SF')
kmf_grade.plot(ax=ax,color='darkslategray')
plt.ylim(0,1.05)
plt.ylabel('Survival Distribution Function')
plt.xlabel('Tenure Months')

In [ ]:
sf_codes

In [ ]:
ma = empltbl3.GRADE.isin(ma_codes)
print sum(ma), ma_codes

In [ ]:
fig = plt.figure(figsize=(12.5,5))

kmf_grade=KaplanMeierFitter()

kmf_grade.fit(empltbl3[ma].Tenure_months, timeline=tenure_month_range, event_observed=empltbl3[ma].status,label='MA')
#grade_sf_df = pd.DataFrame(kmf_grade.survival_function_)
ax = fig.add_subplot(111)
kmf_grade.plot(ax=ax,c='darkred')
kmf_grade.fit(empltbl3[~ma].Tenure_months, timeline=tenure_month_range,event_observed=empltbl3[~ma].status,label='non MA')
kmf_grade.plot(ax=ax,color='darkslategray')
plt.ylim(0,1.05)
plt.ylabel('Survival Distribution Function')
plt.xlabel('Tenure Months')

In [ ]:
pa = empltbl3.GRADE.isin(pa_codes)
print sum(pa), pa_codes

In [ ]:
grade_sf_df.columns

In [ ]:
### More directly look at these 20 groupings:
# create a dataFrame of these survival functions.


fig = plt.figure(figsize=(12.5,5))
#figsize(16,8)

#grade_groups = #list(empltbl3.GRADE.value_counts()[:85].index)
kmf_grade=KaplanMeierFitter()

kmf_grade.fit(empltbl3.Tenure_months, timeline=tenure_month_range, event_observed=empltbl3.status,label='ALL')
gp_grade_sf_df = pd.DataFrame(kmf_grade.survival_function_)
ax=fig.add_subplot(111)
kmf_grade.plot(ax=ax,c='darkslategray')
for g in grade_groupings_list_A:
    
    #plt.figure(figsize=(20,10))
    gem_grade = empltbl3.GRADE.isin(g)
    kmf_grade.fit(empltbl3[gem_grade].Tenure_months, timeline=tenure_month_range,event_observed=empltbl3[gem_grade].status)#,label=grade)
    #if grade.startswith('MG'):
    kmf_grade.plot(ax=ax)
    gp_grade_sf_df = pd.concat([gp_grade_sf_df,kmf_grade.survival_function_],axis=1)
#kmf_gender.fit(female.Tenure_months,  timeline=tenure_month_range,event_observed=female.status)
#kmf_gender.plot(ax=ax, c='deeppink',label='Female')

plt.ylim(0,1.05)
plt.ylabel('Survival Distribution Function')
plt.xlabel('Tenure Months')

#print sum(empltbl3.GRADE.isin(g)), g

In [ ]:
gp_grade_sf_df.columns=['ALL','MG','C','MA','PA','PB','SF','RA','RD','RE','RB','RC','PC','L','FA', 'U', 'B', 'NON','M','R*','OTHR']#,inplace=True)
                               #mg_codes,clvl_codes,ma_codes,pa_codes,pb_codes,sf_codes, ra_codes,rd_codes,re_codes, rb_codes, rc_codes,
                          #pc_codes,l_codes,fa_codes, u_codes,b_codes,non_codes,m_codes,rx_codes,oth_codes

gp_grade_sf_df.head()

In [ ]:
gp_grade_sf_df.iloc[1].plot(kind='bar',color='steelblue')
plt.title('Survival Propensity at 1 year')
plt.ylabel('Survival Distribution')
plt.xlabel('Grouped Grade code')

In [ ]:
np.where(gp_grade_sf_df.ALL.values <=0.50)[0][0]

In [ ]:
gp_grade_sf_df[gp_grade_sf_df['ALL']<=0.50].index[0]/12.


In [ ]:
### Function to return the time when distribution falls to 50%
def find_Npct_survival(x,npct=0.50):
    first_index = np.where(x <=npct)[0][0]
    #ten_years = 
    print first_index, x.iloc[first_index], x.index[first_index]#.timeline
    #first_index-1
    #, x.iloc[first_index-1]
    return x.index[first_index]

In [ ]:
gp_grade_sf_df['ALL'].index[12]

In [ ]:
cnames = []
fiftyPCT = []
for c in gp_grade_sf_df.columns:
    cnames.append(c)
    fiftyPCT.append(find_Npct_survival(gp_grade_sf_df[c])/12.)#.apply(lambda x: find_Npct_survival(x))

In [ ]:
xvals= xrange(len(cnames))
h = plt.bar(xvals,fiftyPCT,color='steelblue')
plt.ylabel('Number of Tenure Years where SF <= 50%')
#plt.xlim(0,21)
plt.xlabel('Grouped Grades')
plt.title('Job Grades impact on SF fraction.')

xticks_pos = [0.65*patch.get_width() + patch.get_xy()[0] for patch in h]
plt.xticks(xticks_pos,cnames,rotation='vertical')
plt.grid(True)
plt.xlim(0,21)

## TRY To look at how age effects sF
* bin this?

In [ ]:
empltbl3.Age_years.hist(bins=70,range=[15,84],color='darkorchid')

In [ ]:
empltbl3.Age_years.describe()

In [ ]:
empltbl3.Age_years.head()

In [ ]:
empltbl3['AgeGroup'] = pd.cut(empltbl3.Age_years,[0,24.5,34.5,44.5,54.5,64.5,100],labels=['0','1','2','3','4','5'])

In [ ]:
age_group_labels = ['0','1','2','3','4','5']
fig = plt.figure(figsize=(12.5,5))
kmf_agp = KaplanMeierFitter()
kmf_agp.fit(empltbl3.Tenure_months, timeline=tenure_month_range, event_observed=empltbl3.status,label='ALL')
    
age_gp_sf_df = pd.DataFrame(kmf_agp.survival_function_)
ax=fig.add_subplot(111)
kmf_agp.plot(ax=ax,c='darkslategray')

for ag in age_group_labels:
    age_group = empltbl3[empltbl3.AgeGroup == ag]
    print ag, len(age_group)
    
    
        
    kmf_agp.fit(age_group.Tenure_months, timeline=tenure_month_range,event_observed=age_group.status,label=ag)
    
    kmf_agp.plot(ax=ax)
    age_gp_sf_df = pd.concat([age_gp_sf_df,kmf_grade.survival_function_],axis=1)

plt.ylim(0,1.05)
plt.ylabel('Survival Distribution Function')
plt.xlabel('Tenure Months')
plt.title('Survival for various age groups')

#print sum(empltbl3.GRADE.isin(g)), g

In [ ]:
age_gp_sf_df.columns = ['ALL','0to25','25to35','35to45','45to55','55to65','65to100']#+=age_group_labels
age_gp_sf_df.head()

In [ ]:
age_grp_types = empltbl3.AgeGroup.unique()
kmf= KaplanMeierFitter()
fig = plt.figure(figsize=(16,9))

for i,agp_type in enumerate(age_grp_types):
    ax = plt.subplot(2,3,i+1)
    ix = empltbl3['AgeGroup'] == agp_type
    kmf.fit( empltbl3[ix].Tenure_months, empltbl3[ix].status, label=age_gp_sf_df.columns[i+1])
    kmf.plot(ax=ax, legend=False)
    plt.title(age_gp_sf_df.columns[i+1])
    plt.xlim(0,500)
    if i==0:
        plt.ylabel('Frac. Working after $n$ months')
    if i == 3:
        plt.xlabel("Tenure in Months")
plt.tight_layout()

In [ ]:
sdf[sdf.Column.isin(cols_to_categorical)]

## this was descriptive, how to make it predictive?

Try using `AalenAdditiveFitter`

In [ ]:
from lifelines import AalenAdditiveFitter
import patsy

In [ ]:
X = patsy.dmatrix('Age_years + COMPANY + INTERN + SEX',empltbl4, return_type='dataframe')
X.columns


In [ ]:
aaf = AalenAdditiveFitter(penalizer=1.0, fit_intercept=True)
X['T'] = empltbl4['Tenure_years']
X['E'] = empltbl4['status']

In [ ]:
aaf.fit(X,'T','E')

In [ ]:
aaf.plot()

In [ ]:
aaf.cumulative_hazards_.loc[40:]

In [ ]:
aaf.

In [ ]:
#X = patsy.dmatrix('SEX + AgeGroup + INTERN + DIVISION_CODE_SFI -1', empltbl3, return_type='dataframe')
X = patsy.dmatrix('AgeGroup + INTERN -1', empltbl3, return_type='dataframe')
X.columns

In [ ]:
aaf = AalenAdditiveFitter(penalizer=1.0, fit_intercept=True)
X['T'] = empltbl3['Tenure_years']
X['E'] = empltbl3['status']

In [ ]:
aaf.fit(X,'T','E')

In [ ]:
aaf.cumulative_hazards_.head()

### Plot the Estimates of cumulative $\beta_i$

In [ ]:
aaf.plot( columns=[ 'AgeGroup[0]','AgeGroup[1]','AgeGroup[2]','INTERN[T.Y]', 'baseline' ], ix=slice(1,35) )

## try to predict someone's tenure

In [ ]:
ix = (empltbl3['AgeGroup'] == '0')
ib = (empltbl3['AgeGroup'] == '2')
#harper = X[ix,:][-1,:][None,:]
#harper[0,-1] = 2003
#print "Harper's unique data point"

In [ ]:
X[ix].iloc[[0,2],0:-2]

In [ ]:
X[ib].iloc[20:25]#,0:-2]

In [ ]:
test1 = X[ix].iloc[0] 
test2 = X[ix].iloc[2]

In [ ]:
test2.values

In [ ]:
aaf.predict_survival_function(X[ix].iloc[[0,2],0:-2]).plot()
plt.xlim(0,10)
aaf.predict_survival_function(X[ib].iloc[[20,21],0:-2]).plot()
plt.xlim(0,10)

In [ ]:
fig = plt.figure(figsize=(12,5))
ax = fig.add_subplot(2,1,1)
aaf.predict_cumulative_hazard(test1.values).plot(ax=ax)

In [ ]:
[x for x in sdf.Column if x.startswith('JOB')]

In [ ]:
sdf[sdf.Column == 'GRADE']

In [ ]:
current.GRADE.value_counts()

# Construct a graph out of the active employees
* use the index from the dataframe as the node number

In [ ]:
import networkx as nx
G=nx.DiGraph()


In [ ]:
#initialize with the ceo index
#initialize with the ceo index
#initialize with the ceo index
ceo_index = list(current[current.KEY==ceo_key].index)[0]
print ceo_index,ceo_key
G.add_node(ceo_index)
G.graph

In [ ]:
def id_dependents(

In [ ]:
# now apply some logic to these columns for keeping and not
columns_to_fix_missing = []
columns_to_fix_NY = []
columns_to_omit = []

In [ ]:
columns_to_fix_missing

In [ ]:
len(sdf[sdf['datatype']=='object'])

In [ ]:
sdf[(sdf['datatype']=='object') & (sdf['x_missing']<=missing_threshold) & (sdf['x_missing'] > 0)]

In [ ]:
empltbl3[empltbl3.STATE == 'NB'][['status','LOC_STATE','LOC_CITY','JOB_FUNCTION']]#,'zip5']]

In [ ]:
sdf.head()

In [ ]:
empltbl[empltbl.POSTAL_SFI.isnull()].zip5

In [ ]:
empltbl.ACTRES1.tail()

In [ ]:
# omit those that have more than 40% missing:
columns_to_omit = sdf[sdf['x_missing'] > missing_threshold].Column.values
print len(columns_to_omit)
columns_to_omit

In [ ]:
pd.crosstab(empltbl['HUBIND'],empltbl['SUPV_DIFF_LOC'])

In [ ]:
empltbl.FLSA_STATUS.value_counts()

In [ ]:
empltbl.JOB_FAMILY.value_counts()[empltbl.JOB_FAMILY.value_counts()>100]

In [ ]:
sort(columns_to_keep)

In [ ]:
reduced_tbl = empltbl[columns_to_keep].copy()
reduced_tbl.shape

In [ ]:
## drop those that are not VOL or NOT in VOLINVOL column
sum(reduced_tbl.isin({'VOLINVOL':['NOT','VOLUNTARY']}))

In [ ]:
for cat_col in cols_to_categorical:
    reduced_tbl[cat_col] = reduced_tbl[cat_col].astype('category')


In [ ]:
reduced_summary_df = summarize_dataframe2(red_tbl)

In [ ]:
reduced_tbl.head()

In [ ]:
reduced_tbl.columns

In [ ]:
pd.crosstab(reduced_tbl['REMOTE'],reduced_tbl['HUBIND'])

In [ ]:
pd.crosstab(reduced_tbl['FTPTCNT1'],reduced_tbl['PTFTCNT1'])

In [ ]:
reduced_tbl.FULL_PART_TIME.value_counts()

In [ ]:
## further reduce by removing
FULL_PART_TIME, ACTRES1, KEY, 

In [ ]:
len(reduced_tbl.JOB_FAMILY.unique()), len(reduced_tbl.JOB_FUNCTION.unique()), len(reduced_tbl.GRADE.unique()),len(reduced_tbl.JOBCODE.unique())

In [ ]:
reduced_tbl.JOB_FUNCTION.value_counts().plot(kind='bar',color='burlywood')

In [ ]:
reduced_tbl.GRADE.value_counts().plot(kind='bar',color='burlywood')

In [ ]:
#sum(reduced_tbl.MAX_RT_ANNUAL==0)
reduced_tbl[reduced_tbl.ANNUAL_RT!=0].ANNUAL_RT.hist(bins=40,color='darkorchid')

In [ ]:
sum(reduced_tbl.ANNUAL_RT==0)

In [ ]:
reduced_tbl['JOB_FUNCTION'].unique()

In [ ]:
initial_columns_list = ['SEX','INTERN','FULL_PART_TIME']#,'COMPANY','VOLINVOL']#'JOB_FUNCTION','COMPANY','HUBIND','VOLINVOL']
sum(reduced_tbl[initial_columns_list].isnull())

In [ ]:
jf_hold = reduced_tbl.JOB_FUNCTION.astype('object')
jf_hold.fillna('XXX',inplace=True)
reduced_tbl['JOB_FUNCTION'] = jf_hold
reduced_tbl['JOB_FUNCTION'] = reduced_tbl['JOB_FUNCTION'].astype('category')

In [ ]:
initial_columns_list = ['SEX','Age_years','INTERN','FULL_PART_TIME','JOB_FUNCTION','COMPANY','HUBIND','VOLINVOL']
sum(reduced_tbl[initial_columns_list].isnull())

In [ ]:
#pull out the reduced columns
fill_flag = True
reduced2 = reduced_tbl[initial_columns_list].copy()
for c in reduced2.columns:
    nmissing = sum(reduced2[c].isnull())
    datatype = reduced2[c].dtype
    if nmissing > 0 and fill_flag:
        if datatype=='category':
            reduced2[c].fillna('XNA',inplace=True)
    
        
    print c, reduced2[c].dtype

In [ ]:
reduced2 = reduced_tbl[initial_columns_list].copy()
reduced2.shape

In [ ]:
dummy_columns_list1 = list(reduced2.columns) #['SEX','INTERN','FULL_PART_TIME','JOB_FUNCTION','COMPANY','HUBIND','VOLINVOL']

In [ ]:
# create a dummified dataframe.
reduced_dummies = pd.get_dummies(reduced2[dummy_columns_list1])
print reduced_dummies.shape    

In [ ]:
dummy_column_list2 = list(reduced_dummies.columns)

In [ ]:
dummy_col_to_drop = []
for col in dummy_columns_list1:
    dclist = [x for x in dummy_column_list2 if x.startswith(col)]
    print dclist
    dummy_col_to_drop.append(dclist[0])

dummy_col_to_keep = list(set(dummy_column_list2) - set(dummy_col_to_drop))

reduced3 = reduced_dummies[dummy_col_to_keep].copy()
#add the non-categorical back in
#reduced3[initial_columns_list[1]] = reduced2[initial_columns_list[1]]
reduced3.shape

In [ ]:
## add in status and time
reduced3['Age'] = empltbl['Age_years']
reduced3['status'] = empltbl['status']
reduced3['Tenure'] = empltbl['Tenure_months']

## Aalen's Additive model

In [ ]:
from lifelines import AalenAdditiveFitter
reduced3.head()

In [ ]:
reduced3.columns

In [ ]:
aaf_tenure = AalenAdditiveFitter(penalizer=1.0,fit_intercept=True)

In [ ]:
aaf_tenure.fit(reduced3,'Tenure',event_col='status')

In [ ]:
aaf_tenure.plot(columns=['FULL_PART_TIME_P','Age','SEX_M','baseline'])

In [ ]:
aaf_tenure_b = AalenAdditiveFitter(penalizer=1.0,fit_intercept=True)

In [ ]:
aaf_tenure_b.fit(reduced3,'Tenure',event_col='status')

## CoxPH

In [ ]:
from lifelines import CoxPHFitter

In [ ]:
cf_tenure = CoxPHFitter()
cf_tenure.fit(reduced3,'Tenure',event_col='status')


In [ ]:
cf_tenure.summary()

In [ ]:
cf_tenure.plot(columns=['FULL_PART_TIME_P','Age','SEX_M','baseline'])

In [ ]:
cf_tenure.hazards_

In [ ]:
cf_tenure.baseline_cumulative_hazard_.plot()

In [ ]:
cf_tenurereenure.base

In [ ]:
reduced3.index = np.arange(0,len(reduced3))

In [ ]:
# k-fold

from lifelines.utils import k_fold_cross_validation

#regression_dataset = load_regression_dataset()
cf = CoxPHFitter()
scores = k_fold_cross_validation(cf, reduced3, 'Tenure', event_col='status', k=5)
print scores
print scores.mean()
print scores.std()

In [ ]:
import patsy

In [ ]:
X = patsy.dmatrix('SEX + JOB_FUNCTION + Age_years + INTERN + COMPANY + VOLINVOL + HUBIND - 1', reduced2, return_type='dataframe')

In [ ]:
patsy.__version__

In [ ]:
# now use lifelines
from lifelines import KaplanMeierFitter

In [ ]:
figsize(14,6)
kmf = KaplanMeierFitter()
kmf.fit(empltbl.Tenure_months, event_observed=empltbl.status)
kmf.plot()
plt.ylabel('Survival Distribution Function')
plt.xlabel('Tenure months')
plt.title('KaplanMeier Survival: ALL employees')

In [ ]:
kmf.survival_function_

### look at difference between M & F

In [ ]:
male = empltbl[empltbl['SEX'] =='M']
female = empltbl[empltbl['SEX'] =='F']
len(male),len(female)

In [ ]:
male.Tenure_months.describe(), female.Tenure_months.describe()

In [ ]:
min_tenure_months= empltbl.Tenure_months.min()
max_tenure_months = empltbl.Tenure_months.max()
print min_tenure_months, max_tenure_months
tenure_month_range = np.linspace(min_tenure_months,max_tenure_months,65)
len(tenure_month_range)

In [ ]:
male.Tenure_months.hist(normed=True,color='dodgerblue',bins=65,label='M')
female.Tenure_months.hist(normed=True,color='deeppink',alpha=0.5,bins=65,label='F')
plt.xlabel('Tenure_months')
plt.ylabel('Normed Distribution')
plt.legend()

In [ ]:
kmf_gender = KaplanMeierFitter()
#plt.figure(figsize=(20,10))
kmf_gender.fit(male.Tenure_months, timeline=tenure_month_range,event_observed=male.status)
ax = kmf_gender.plot(c='dodgerblue',label='Male')
kmf_gender.fit(female.Tenure_months,  timeline=tenure_month_range,event_observed=female.status)
kmf_gender.plot(ax=ax, c='deeppink',label='Female')
ylim(0,1.05)
plt.ylabel('Survival Distribution Function')
plt.xlabel('Tenure Months')

## Hazard Function

In [ ]:
from lifelines import NelsonAalenFitter
#plt.figure(figsize=(12.5,8))
naf = NelsonAalenFitter()
naf.fit(male.Tenure_months,timeline=tenure_month_range, event_observed=male.status)
ax = naf.plot(c='dodgerblue',label='Male')
naf.fit(female.Tenure_months,  timeline=tenure_month_range,event_observed=female.status)
naf.plot(ax=ax, c='deeppink',label='Female')
#ylim(0,1.05)
plt.ylabel('Hazard Function')
plt.xlabel('Tenure Months')
#fit(male.Tenure_months,timeline=tenure_month_range, event_observed=male.status)

### Repeat using age in years as timeline

In [ ]:
empltbl.Age_years.describe()

In [ ]:
sum(empltbl.Age_years > 80)

In [ ]:
min_age = empltbl.Age_years.min()
max_age = empltbl.Age_years.max()
print min_age, max_age, max_age - min_age
delta_age= int(max_age - min_age)
age_range = np.linspace(min_age,max_age,delta_age)
len(age_range)

In [ ]:
kmf_age = KaplanMeierFitter()
kmf_age.fit(empltbl.Age_years, timeline=age_range,event_observed=empltbl.status)
kmf_age.plot(color='darkgoldenrod')
plt.ylabel('Survival Distribution Function')

In [ ]:
male.Age_years.hist(normed=True,color='dodgerblue',bins=82,alpha=0.7,label='M')
female.Age_years.hist(normed=True,color='deeppink',alpha=0.3,bins=82,label='F')
plt.xlabel('Age_years')
plt.ylabel('Normed Distribution')
plt.legend()

In [ ]:
kmf_gender_age = KaplanMeierFitter()
#plt.figure(figsize=(20,10))
kmf_gender_age.fit(male.Age_years, timeline=age_range,event_observed=male.status)
ax = kmf_gender_age.plot(c='dodgerblue',label='Male')
kmf_gender_age.fit(female.Age_years,  timeline=age_range,event_observed=female.status)
kmf_gender_age.plot(ax=ax, c='deeppink',label='Female')
ylim(0,1.05)
plt.ylabel('Survival Distribution Function')
plt.xlabel('Age (years)')
plt.title('Gender-split KM')

## Cox Proportional Hazard Regression


In [ ]:
from lifelines import CoxPHFitter

In [ ]:
plt.scatter(empltbl.FLOORCNT1,empltbl.DEPTCNT1,color='burlywood',alpha=0.3)

In [ ]:
#pd.crosstab(empltbl, index = 'JOBCNT1',columns =
#(empltbl, 
empltbl[['JOBCNT1','LOCCNT1','DEPTCNT1','GRADECNT1','FLOORCNT1','SUPVCNT1']].corr()

In [ ]:
c='EMPL_CLASS'
empltbl[c].value_counts()

In [ ]:

len(empltbl[c].unique())

In [ ]:
plt.scatter(empltbl.ANNUAL_RT,empltbl.SAL1,color='burlywood',alpha=0.3)
empltbl[['ANNUAL_RT','SAL1']].corr()

In [ ]:
len(empltbl[empltbl.ANNUAL_RT>0])

In [ ]:
np.log(empltbl.ANNUAL_RT+1).hist(bins=30)

In [ ]:
empltbl.SAL1.hist(bins=80)
empltbl.SAL1.describe()

In [ ]:
sum(empltbl.SAL1 == 0.0)

In [ ]:
empltbl.RELOCATE_ALL_SFI.value_counts()

In [ ]:
empltbl.LOCATION.value_counts().hist(bins=50)

## encode some of the categorical columns

## Per discussion with HR TEAM on April 22, 2015

In [ ]:
empl = empltbl4[empltbl4.sep_status<=2] # Remove the OTHER & UNKNOWN reasons

In [ ]:
empl.shape

In [ ]:
# look at the list of ACTRES1 for  ReTIREMENT
uniq_action_reasons_1 = empl.ACTRES1.unique()
print len(uniq_action_reasons_1)
#ret_action_reasons_1 = [x for x in uniq_action_reasons_1 if 'RETIREMENT' in x]
#print len(ret_action_reasons_1)

In [ ]:
temp_list = [x.split(';') for x in uniq_action_reasons_1]
from itertools import chain
act_reason_1_list = list(chain.from_iterable(temp_list))
print len(act_reason_1_list)

In [ ]:
act_reason_1_set = set(act_reason_1_list)

In [ ]:
len(act_reason_1_set)

In [ ]:
[x for x in act_reason_1_set if 'DISABI' in x]

In [ ]:
#possible_retire_codes = 
possible_retire_codes = [x for x in act_reason_1_set if ('RET' in x and  'RETURN' not in x) ]
possible_retire_codes.append('DISABILITY')
len(possible_retire_codes)

In [ ]:
possible_retire_codes.sort()

In [ ]:
possible_retire_codes

In [ ]:
t1 = empl.ACTRES1.ix[1]
[a for a in t1.split(';') if a in possible_retire_codes ]

In [ ]:
possible_retire_codes

In [ ]:
empl['disabled'] = empl.ACTRES1.apply(lambda x: len([a for a in x.split(';') if a == 'DISABILITY']))
empl[empl['disabled']!=0].Age_years.hist()

In [ ]:
sum(empl.disabled==1)

In [ ]:
empl.sep_status.value_counts()

In [ ]:
len(empl), len(empltbl4)

In [ ]:
empl[empl['retired']==1].Age_years.hist(bins=70,range=[16,85],color='darkorchid')
empl[empl['retired']==0].Age_years.hist(bins=70,range=[16,86],color='forestgreen',alpha=0.7)

In [ ]:
sum(empl.retired==1)

In [ ]:
pd.crosstab(empl.retired,empl.sep_status)

In [ ]:
empl[(empl.sep_status==0) & (empl.retired==1)]['KEY']

In [ ]:
print empltbl[empltbl.KEY=='185980322857378'][date_columns]
empltbl[empltbl.KEY=='185980322857378'].Age_years

In [ ]:
empltbl[empltbl.KEY=='185980322857378'][['ACTRES1','ACTRES2','ACTRES3','ACTRES4','ACTRES5','ACTRES6','ACTRES7','ACTRES9','ACTRES10']].values

In [ ]:
empl[empl['retired']==1].Age_years.hist(bins=70,range=[16,85],color='darkorchid',label='retired')
empl[(empl['retired']==0) & (empl.status==1)].Age_years.hist(bins=70,range=[16,86],color='forestgreen',alpha=0.7,label='separated')
empl[(empl['retired']==0) & (empl.status==0)].Age_years.hist(bins=70,range=[16,86],color='darkkhaki',alpha=0.7,label='current')
#empl[(empl['retired']==0) & (empl.status==1)].Age_years.hist(bins=70,range=[16,86],color='forestgreen',alpha=0.7)
plt.ylabel('Number of employees')
plt.xlabel('Age')
plt.legend()

In [ ]:
sum(empl.SAL1== 0.0)

In [ ]:
len(empl)

## April 27, 2015 
* need to categorize all data I can before loading into R

In [ ]:
empltbl4=pd.read_csv('employee_dataframe4.tsv',sep='\t',index_col=0,dtype={'EMPL_CLASS':np.str,'EMPL_TYPE':np.str})
empltbl4.shape

In [ ]:
sdf4 = summarize_dataframe2(empltbl4)

In [ ]:
sdf4[sdf4.datatype=='object']

In [ ]:
# convert ethnic_group to int
empltbl4['ETHNIC_GROUP'] = empltbl4['ETHNIC_GROUP'].apply(lambda x: int(x))

In [ ]:
empltbl4.STATE.value_counts()[empltbl4.STATE.value_counts()<50]

In [ ]:
empltbl4.LOC_STATE.value_counts()[empltbl4.LOC_STATE.value_counts()<50]

In [ ]:
empltbl4[empltbl4.STATE=='HH'][['status','LOC_STATE','STATE','KEY']]#in [11,29]]

In [ ]:
empltbl[empltbl.STATE=='BC'][['KEY','LOC_STATE','STATE','status','POSTAL_SFI','ADDRESS1','ADDRESS2','TERMINATION_DT','GRADE','LOCATION']]

In [ ]:
empltbl[empltbl.KEY=='811710100533306'].values
#empltbl[empltbl.KEY=='296979068916747'].values

In [ ]:
empltbl[empltbl.KEY=='811710100533306'][['ADDRESS1','ADDRESS2','LOCATION','POSTAL_SFI']]

In [ ]:
empltbl4.SEX.value_counts()
empltbl4['COMP_FREQ_ANNUAL'] = 1
empltbl4.loc[empltbl4['COMP_FREQUENCY']=='H','COMP_FREQ_ANNUAL'] = 0
#empltbl4.drop('COMP_FREQUENCY',axis=1,inplace=True)

# now deal with male,female
empltbl4.replace({'SEX':{'M': 1,'F':0}},inplace=True)
empltbl4[['COMP_FREQ_ANNUAL','SEX','SHIFT']].head()

In [ ]:
empltbl4.COMP_FREQ_ANNUAL.value_counts()

In [ ]:
empltbl4.drop('COMP_FREQUENCY',axis=1,inplace=True)
empltbl4.drop('STATE',axis=1,inplace=True)
empltbl4.shape


In [ ]:
# assign 1 to Y, 0 to N for the following columns
columns_to_fix_NY = ['INTERN','FULLPART1','RELOCATE_ALL_SFI','HUBIND','REMOTE','REMOTE_SUPV','SUPV_DIFF_LOC','PARTFULL1']
for acol in columns_to_fix_NY:
    empltbl4.replace({acol :{'Y':1,'N':0}},inplace=True) 

In [ ]:
cols_to_cat = ['FLSA_STATUS','FULL_PART_TIME','SHIFT','EMPL_TYPE','COMPANY','ETHNIC_GROUP','EEO1CODE','EMPL_CLASS','JOB_FUNCTION']

In [ ]:
sdf4.sort(['arity','datatype'],ascending =True)[sdf4.arity > 9]

In [ ]:
## convert HIRE_DT to HIRE_YEAR
[c for c in empltbl4.columns if c.endswith('DT')]

In [ ]:
empltbl4['retired'] = empltbl4.ACTRES1.apply(lambda x: identify_retired(x))
sum(empltbl4.retired==1)

In [ ]:
# encode the VOLINVOL column as ['INVOLUNTARY':2,'VOLUNTARY':1,'NOT':0,'
empltbl4['sep_status'] = empltbl4['VOLINVOL'].replace({'NOT':0,'VOLUNTARY':1,'INVOLUNTARY':2,'OTHER':3,'UNKNOWN':3})
empltbl4.sep_status.value_counts()

In [ ]:
empl = empltbl4.copy()

In [ ]:
empl = empltbl4.copy()
empl.drop(['VOLINVOL','ACTRES1','HIRE_DT'],axis=1,inplace=True)
empl.head()

In [ ]:
egrp, bc1 = create_factorized_df(empl,acol)

In [ ]:
sgrp, bc1 = create_factorized_dfrized_df(empl,'LOC_STATE')

In [ ]:
more_cols_to_cat = ['LOC_STATE','GRADE','EXT_FUNC_ID_SFI','JOB_FAMILY']

In [ ]:
## remove sep_status == 3
empl=empl[empl.sep_status<3].copy()
empl.shape

In [ ]:
for acol in more_cols_to_cat[:2]:
    print acol
    t_df,bc1 = create_factorized_df(empl,acol)
    # drop the original column
    empl.drop(acol,inplace=True,axis=1)
    # append the factorized categories
    empl = pd.concat([empl,t_df],axis=1)
    base_category_list.append(bc1)


In [ ]:
empl.shape

In [ ]:
## save this to a file
empl.drop(more_cols_to_cat[2:],axis=1,inplace=True)
empl.drop('KEY',axis=1,inplace=True)
empl.to_csv('employee_dataframe5.tsv',sep='\t',index =False)

#### create integers for times; required for CoxPH in H2O.

In [ ]:
empl[['Age','Tenure']] = empl[['Age_years','Tenure_years']].applymap(lambda x: int(round(x,0)))

In [ ]:
empl[['Age','Age_years']].head()

In [ ]:
empl.to_csv('employee_dataframe6.tsv',sep='\t',index =False)

In [ ]:
sdfB = summarize_dataframe2(empl)

In [ ]:
sdfB.sort('arity')

In [ ]:

#empl.FUNC_ID_SFI.value_counts()

In [ ]:
def convert_to_int(x):
    try:
        x=int(x)
    except:
        x=-1
    return x

empl.FUNC_ID_SFI = empl.FUNC_ID_SFI.apply(lambda x: convert_to_int(x))
len(empl.FUNC_ID_SFI.unique())

In [ ]:
len(empl)

In [ ]:
empl.drop(['LOCATION','LEGACY_DEPT_SFI','Tenure_months'],axis=1,inplace=True)

In [ ]:
empl.to_csv('employee_dataframe5.tsv',sep='\t',ignore_index =True)

In [ ]:
[c for c  in enumerate(list(empl.columns))]

In [ ]:
empl.loc[:2000,'status'].value_counts()

In [ ]:
empl.loc[:2000,'retired'].value_counts()

In [ ]:
sum(empl.retired==1)

In [ ]:
empl.loc[:4000,'retired'].value_counts()


In [ ]:
#X = patsy.dmatrix('Age_years + COMPANY + INTERN + SEX',empltbl4, return_type='dataframe')
#X.columns
aaf = AalenAdditiveFitter(penalizer=1.0, fit_intercept=True)
X['T'] = empltbl4['Tenure_years']
X['E'] = empltbl4['status']

In [ ]:
empl.Age_years

In [ ]:
Xft = patsy.dmatrix( 'Age_years +ANNUAL_RT+PERF1+INTERN+SEX',empl,return_type='dataframe')
Xft.shape

In [ ]:
Xft['T'] = empl.Tenure_years#empl.ix[:4010]['Tenure_years']
Xft['E']=empl.retired#empl.ix[:4010]['retired']

In [ ]:
aaf_ret = AalenAdditiveFitter(penalizer=1.0, fit_intercept=True)
aaf_ret.fit(Xft,'T','E')

In [ ]:
aaf_ret.plot(columns=['baseline','Age_years','INTERN','SEX','ANNUAL_RT'],ix=slice(1,15))

In [ ]:
from lifelines import CoxPHFitter
cf = CoxPHFitter()

In [ ]:
%%time 
cf.fit(Xft[Xft.columns[1:]],'T','E')

In [ ]:
cf.summary()

In [ ]:
few_columns1=['Age_years','ANNUAL_RT','PERF1','INTERN','SEX','Tenure_years','retired']
reduced1 = empl.loc[:,few_columns1].copy()
reduced1.shape

In [ ]:
%%time 
cf.fit(reduced1,'Tenure_years','retired')

In [ ]:
cf.summary()

In [ ]:
cf.confidence_intervals_.head()

In [ ]:
cf.hazards_.head()

In [ ]:
sum(aaf_ret.event_observed)

In [ ]:
from lifelines.utils import k_fold_cross_validation
xv_cf = CoxPHFitter()
cf_scores = k_fold_cross_validation(xv_cf, reduced1,duration_col='Tenure_years',event_col='retired',k=10)

In [ ]:
cf_scores.mean(), cf_scores.std()

In [ ]:
xv_cf.hazards_

In [ ]:
reduced1.ix[44]

In [ ]:
np.shape(reduced1.iloc[4:5,:-2].values)
retired_sf_avg = xv_cf.predict_survival_function(reduced1[reduced1.retired==1][reduced1.columns[:-2]].values).mean(axis=1)#.plot()#.iloc[34:35,:-2].values).plot()

In [ ]:
notretired_sf_avg = xv_cf.predict_survival_function(reduced1[reduced1.retired==0][reduced1.columns[:-2]].values).mean(axis=1)

In [ ]:
xv_cf.predict_survival_function(re

In [ ]:
retired_sf_avg.plot(label='retired')
notretired_sf_avg.plot(label='not-retired')
plt.xlabel('Tenure Years')
plt.ylabel('Predicted Survival Function')

In [ ]:
reduced1.iloc[24:25,:-2]

In [ ]:
cf.baseline_survival_.plot()

## examining relationships (correlations) between columns

In [ ]:
empl[['Tenure_years','hire_year']].corr()

In [ ]:
empl.plot(kind='scatter',x='Tenure_years',y='hire_year',
          alpha=0.3)

In [ ]:
empl.plot(kind='scatter',x='hire_year',y='Age_years',
          alpha=0.3)

In [ ]:
[x for x in empl.columns if x.endswith('MOS')]

In [ ]:
empl.plot(kind='scatter',x='CUR_LOC_MOS',y='CUR_DEPT_MOS',color='chartreuse',          alpha=0.3)

In [ ]:
pd.crosstab(empl.CUR_DEPT_MOS,empl.CUR_FUNC_MOS)

In [ ]:
empl.Tenure.describe()


In [ ]:
empl.Tenure.hist(bins=65,color='steelblue')

## EXPLORE more about some of these features.
* especially ones that may be missing or zeroed out
* SAL1, MIN_RT_ANNUAL

In [ ]:
empl.MIN_RT_ANNUAL.describe()

In [ ]:
sum(empl.MIN_RT_ANNUAL==0.0), sum(empl.MAX_RT_ANNUAL==0.0)

In [ ]:
empl[empl.MIN_RT_ANNUAL==0.0]['Tenure'].hist(range=[0,64],normed=True,bins=65)
empl[empl.MIN_RT_ANNUAL!=0.0]['Tenure'].hist(range=[0,64],normed=True,bins=65,alpha=0.6)

In [ ]:
empl[empl.MIN_RT_ANNUAL>0].MIN_RT_ANNUAL.hist(bins=30)

In [ ]:
empl[empl.MIN_RT_ANNUAL>1].MIN_RT_ANNUAL.describe()

In [ ]:
empl[empl.MAX_RT_ANNUAL>1].MAX_RT_ANNUAL.describe()

In [ ]:
print empl.TELE_MOS.describe()
empl.plot(kind='scatter', x='TELE_MOS',y='status')

In [ ]:
[c for c in empl.columns if c.startswith('JOB_FUNCTION')]
#pd.crosstab(empl.GRADE_MA1,empl.JOB_FUNCTION_OTH)

In [ ]:
pd.crosstab(empltbl4.GRADE,empltbl4.JOB_FUNCTION)